# Preprocesamiento

In [1]:
import os
from nltk.stem import SnowballStemmer 
import nltk
import string
import re
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from scipy.spatial.distance import jaccard
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
#Path de los datos 
#data_path = r'C:\Users\kevin\OneDrive\Documentos\GitHub\ProyectoRI\data\training_txt'
#data_path = r'D:\U\7. Septimo\RI\ProyectoRI\data\training_txt'
data_path = r'C:\Users\usuario\Fer-Pc\Escritorio\EPN\2024-A\SEPTIMO_SEMESTRE\RECUPERACION_DE_INFORMACION\repoMantillaRI\ProyectoRI\data\training_txt'

In [3]:
documents = [] #  Vector de documentos
for filename in os.listdir(data_path):
    if filename.endswith('.txt'): 
        path = os.path.join(data_path, filename) #Abrimos cada archivo 
        with open(path, 'r', encoding='utf-8') as file:
            content = file.read() # leemos
            documents.append((filename, content))

In [4]:
#Path de las stopwords
#stopwords_path = r"C:\Users\kevin\OneDrive\Documentos\GitHub\ProyectoRI\data\stopwords.txt"
#stopwords_path = r"D:\U\7. Septimo\RI\ProyectoRI\data\stopwords.txt"
stopwords_path = r"C:\Users\usuario\Fer-Pc\Escritorio\EPN\2024-A\SEPTIMO_SEMESTRE\RECUPERACION_DE_INFORMACION\repoMantillaRI\ProyectoRI\data\stopwords.txt"

In [5]:
#Abrimos el archivo 
with open(stopwords_path, 'r', encoding='utf-8') as file:
    stop_words = set(file.read().splitlines()) # leemos las stopwords

In [6]:
#Stemmer se usa para reducir las palabras a su raíz
stemmer = SnowballStemmer('english')

In [7]:
#Definimos una función que normaliza el texto con todos los requisitos necesarios:
def preprocess_text(text):
    text = text.lower() #conviertimos en minúsculas
    text = re.sub(r'\d+', '', text)  
    text = text.translate(str.maketrans('', '', string.punctuation))#eliminamos los signos de puntuación
    tokens = nltk.word_tokenize(text)#tokenizamos
    processed_tokens = [stemmer.stem(word) for word in tokens if word not in stop_words] #aplicamos stemming
    return ' '.join(processed_tokens)

In [8]:
preprocessed_documents = [(filename, preprocess_text(content)) for filename, content in documents]

# Cargar data importante

In [9]:
querys_ = [
    'rand',
    'income',
    'palmkernel',
    'gas',
    'ship',
    'dmk',
    'jet',
    'cotton-oil',
    'coconut',
    'housing',
    'dlr',
    'soy-oil',
    'lumber'
    'nzdlr',
    'livestock',
    'crude',
    'copra-cake',
    'hog',
    'wpi',
    'lei',
    'naphtha',
    'palladium',
    'propane',
    'rye',
    'jobs',
    'cpi',
    'gold',
    'cocoa',
    'lin-oil',
    'oat',
    'carcass',
    'bop',
    'trade',
    'rubber',
    'groundnut-oil',
    'veg-oil',
    'rapeseed',
    'sun-meal',
    'interest',
    'gnp',
    'cotton',
    'money-fx',
    'orange',
    'earn',
    'l-cattle',
    'yen',
    'rice',
    'sorghum',
    'oilseed',
    'cpu',
    'soy-meal',
    'groundnut',
    'rape-oil',
    'money-supply',
    'sunseed',
    'platinum',
    'nkr',
    'wheat',
    'reserves',
    'coconut-oil',
    'coffee',
    'iron-steel',
    'zinc',
    'potato',
    'acq',
    'silver',
    'nat-gas',
    'alum',
    'nickel',
    'soybean',
    'dfl',
    'lead',
    'meal-feed',
    'retail',
    'instal-debt',
    'pet-chem',
    'tea',
    'copper',
    'ipi',
    'corn',
    'palm-oil',
    'tin',
    'strategic-metal',
    'barley',
    'fuel',
    'heat',
    'castor-oil',
    'sugar',
    'grain',
    'sun-oil',
]

# Vectorizacion

In [10]:
# Extraer solo los contenidos preprocesados para la vectorización
preprocessed_contents = [content for _, content in preprocessed_documents]

In [11]:
# Obtener los nombres de los archivos y el contenido para preprocesar
file_names, preprocessed_documents = zip(*documents)

In [12]:
print(file_names)

('1.txt', '10.txt', '100.txt', '1000.txt', '10000.txt', '10002.txt', '10005.txt', '10008.txt', '10011.txt', '10014.txt', '10015.txt', '10018.txt', '10023.txt', '10025.txt', '10027.txt', '1003.txt', '10032.txt', '10035.txt', '10037.txt', '10038.txt', '10040.txt', '10041.txt', '10042.txt', '10043.txt', '10046.txt', '10048.txt', '10049.txt', '10050.txt', '10052.txt', '10053.txt', '10054.txt', '10057.txt', '10058.txt', '10061.txt', '10062.txt', '10064.txt', '10065.txt', '10066.txt', '10067.txt', '10068.txt', '1007.txt', '10071.txt', '10073.txt', '10074.txt', '10075.txt', '10076.txt', '10078.txt', '10079.txt', '1008.txt', '10080.txt', '10081.txt', '10083.txt', '10085.txt', '10086.txt', '10088.txt', '10089.txt', '1009.txt', '10090.txt', '10091.txt', '10092.txt', '10094.txt', '10095.txt', '10096.txt', '10097.txt', '10098.txt', '10099.txt', '101.txt', '1010.txt', '10100.txt', '10102.txt', '10105.txt', '10106.txt', '10107.txt', '10108.txt', '10109.txt', '1011.txt', '10110.txt', '10112.txt', '10

### BoW 

#### Jaccard

In [13]:
vectorizer_bow = CountVectorizer(binary=True)
X_bow = vectorizer_bow.fit_transform(preprocessed_contents).toarray()
X_querys_BoW = vectorizer_bow.transform(querys_)

In [14]:
# Vectorizar consultas
querys_of_path_dict = {query: 1 for query in querys_} 
X_querys_BoW = vectorizer_bow.transform(list(querys_of_path_dict.keys())).toarray()

In [15]:
def calculate_jaccard_similarities(query_vector, document_vectors):
    similarities = []
    for doc_vector in document_vectors:
        similarity = 1 - jaccard(query_vector, doc_vector)
        similarities.append(similarity)
    return similarities

In [31]:
# Suponiendo que tienes funciones definidas como calculate_jaccard_similarities y X_querys_BoW y X_bow están definidos adecuadamente

# Parámetros
threshold = 0.03
num_relevant_documents = []

# Iterar sobre cada vector de consulta en X_querys_BoW
for query_vector in X_querys_BoW:
    relevant_count = sum(similarity >= threshold for similarity in calculate_jaccard_similarities(query_vector, X_bow))
    num_relevant_documents.append(relevant_count)

# Imprimir resultados
print("Número de consultas evaluadas:", len(num_relevant_documents))
print("Documentos relevantes por consulta:", num_relevant_documents)


Número de consultas evaluadas: 89
Documentos relevantes por consulta: [14, 0, 0, 70, 47, 0, 2, 153, 3, 0, 234, 143, 0, 3, 55, 4, 7, 0, 0, 1, 2, 0, 1, 0, 2, 52, 16, 141, 4, 1, 0, 198, 8, 143, 146, 0, 21, 113, 7, 10, 141, 0, 135, 0, 52, 10, 7, 0, 0, 12, 1, 144, 146, 0, 2, 0, 83, 0, 146, 0, 20, 9, 1, 0, 11, 70, 0, 3, 34, 0, 21, 24, 26, 38, 2, 3, 17, 0, 52, 162, 2, 21, 25, 13, 10, 141, 44, 51, 155]


In [32]:
documentos_relevantes = {
    'acq': ['10', '1000', '10005', '10018', '10025', '10042', '10049', '10052', '10058', '10065', '10066', '10071', '10073', '10078', '10086', '10088', '10098', '10102', '10106', '10128', '10129', '10139', '10146', '10158', '10168', '10171', '10190', '1020', '10213', '10218', '10219', '10224', '10238', '1029', '10310', '10311', '10313', '1032', '10335', '10348', '10361', '10371', '1039', '10399', '10404', '10414', '10419', '10423', '10429', '10434', '10435', '10442', '10446', '10453', '10454', '10458', '10462', '10467', '10473', '10508', '10518', '10527', '10541', '10551', '10555', '10563', '10576', '10582', '10585', '10587', '10596', '10674', '10676', '10692', '10694', '10714', '10721', '1074', '10744', '1076', '10768', '10784', '10786', '10787', '10788', '10794', '10815', '10816', '10831', '10834', '10843', '10844', '10848', '10849', '10867', '10871', '10872', '10874', '10883', '10887', '10892', '10895', '10899', '10901', '10908', '10909', '10918', '10928', '10941', '10950', '10960', '10961', '10962', '10969', '10971', '10978', '10985', '10988', '10989', '1099', '10992', '10994', '10998', '110', '11006', '11029', '1103', '1104', '11041', '11048', '11051', '11056', '11063', '11066', '11074', '11078', '11086', '11088', '11095', '11101', '11102', '11113', '11126', '11133', '11134', '11136', '11139', '11143', '11144', '11161', '11177', '1118', '11192', '11201', '11202', '11210', '11213', '1123', '11236', '11237', '11239', '11242', '11252', '11253', '11255', '11257', '11268', '11274', '11279', '11285', '11289', '11292', '11295', '11299', '11324', '11344', '11348', '11349', '11362', '11369', '11373', '11374', '11378', '11386', '11393', '11396', '11402', '11405', '11431', '11441', '11449', '11464', '11465', '11468', '11469', '11477', '11484', '11488', '11492', '11504', '11505', '11508', '11514', '11515', '11522', '11523', '11526', '11527', '1153', '11539', '11550', '11553', '11563', '11579', '11584', '1159', '11597', '11598', '11599', '11602', '11619', '11623', '1164', '11657', '11661', '11679', '11707', '11722', '11738', '11743', '11744', '11748', '11751', '11755', '11762', '11763', '11790', '11791', '11793', '11808', '11812', '11816', '11821', '11822', '11825', '11829', '11833', '11837', '11841', '11847', '11854', '11860', '11863', '11867', '11871', '11872', '11873', '11876', '11888', '11891', '11892', '11895', '11905', '11909', '11910', '11915', '11916', '11920', '11923', '11954', '11962', '11968', '11981', '11982', '11996', '11997', '11999', '12', '12007', '12018', '12028', '1203', '12032', '12049', '12053', '12054', '12068', '12083', '12087', '1209', '12093', '12094', '12106', '12127', '1213', '12132', '12133', '12138', '12139', '12140', '12146', '12156', '1217', '12171', '12172', '12173', '12178', '12186', '12191', '12195', '1220', '12203', '12205', '12213', '12219', '12221', '12222', '12225', '12226', '12236', '12250', '12276', '1228', '12281', '12298', '12305', '12309', '12321', '12331', '12339', '12344', '12345', '12350', '12356', '12368', '12374', '12378', '1238', '12386', '12390', '12395', '12409', '1243', '12441', '12443', '12449', '12454', '12460', '12468', '12485', '12492', '125', '12504', '12512', '12529', '12530', '12532', '12538', '12540', '12546', '12557', '12751', '12759', '1276', '12767', '12797', '128', '1280', '12822', '12827', '12834', '12839', '12852', '12853', '12858', '12866', '12883', '12888', '12889', '12890', '12900', '12905', '1291', '1292', '12933', '12937', '12956', '12965', '12973', '12974', '12981', '12988', '12995', '12996', '13005', '13011', '13021', '13023', '1303', '13036', '13037', '13041', '13043', '13054', '13060', '13072', '13077', '13085', '13089', '13090', '13095', '13103', '13124', '13139', '13151', '1316', '13163', '13168', '13175', '13182', '13188', '13189', '13194', '13195', '13196', '13197', '13199', '13205', '13206', '13210', '13211', '13212', '13265', '13393', '134', '1340', '135', '13532', '1354', '13704', '13711', '13714', '13753', '13775', '13787', '13790', '13870', '13874', '14028', '1409', '14099', '1412', '14131', '14154', '14156', '14177', '14201', '1425', '14263', '1429', '14298', '14315', '14376', '14394', '14423', '14469', '14485', '14504', '14506', '14542', '14563', '14582', '14587', '14604', '14619', '14623', '14626', '14679', '14715', '14736', '1477', '1480', '14843', '14852', '14865', '14888', '14900', '14907', '14909', '14921', '14932', '14941', '14943', '14949', '1497', '14978', '14982', '1500', '15001', '15004', '15024', '15031', '15037', '15052', '1506', '15060', '15061', '15063', '15069', '15091', '15111', '15119', '1512', '15122', '15124', '15128', '15138', '15146', '15149', '15156', '15161', '15204', '15210', '15217', '15239', '15255', '15276', '15281', '15294', '153', '15302', '15306', '15308', '15311', '15335', '15337', '15349', '15363', '15366', '15372', '15404', '15421', '15425', '15464', '15467', '15468', '15479', '15487', '15501', '15542', '15544', '15545', '15561', '15565', '15571', '15575', '15591', '15595', '15596', '15600', '15605', '15616', '1562', '15629', '15664', '15672', '15682', '15698', '157', '15713', '15717', '15739', '1577', '15778', '15785', '1581', '15810', '1584', '15861', '15879', '15885', '15892', '15894', '15898', '15900', '15912', '15918', '15920', '15956', '15961', '15963', '15968', '15970', '15993', '15995', '15997', '16002', '16007', '1601', '16021', '16026', '16029', '16033', '16037', '16041', '1606', '16091', '16117', '16130', '16136', '16145', '16152', '16155', '16159', '16163', '16173', '16176', '162', '16203', '16210', '16211', '16215', '16216', '1622', '16223', '16232', '16233', '16243', '16247', '16258', '1626', '16277', '1628', '16287', '16302', '16307', '16320', '16332', '1634', '16370', '16379', '16395', '16398', '16402', '16403', '16404', '16406', '16409', '16419', '16420', '16427', '16443', '16444', '16449', '16450', '16457', '16459', '16460', '16463', '16465', '16475', '16479', '16483', '1649', '16493', '16510', '16512', '16513', '16519', '16521', '16523', '16531', '16555', '16563', '16570', '16577', '16583', '1659', '16591', '16597', '16621', '16622', '16646', '16648', '1665', '16652', '16656', '1666', '16663', '16667', '1667', '16675', '16676', '16677', '16680', '16684', '16685', '16688', '16695', '16700', '16707', '16714', '16718', '16727', '1673', '16734', '16737', '16747', '1676', '16761', '1678', '1679', '16796', '16811', '16833', '1684', '16851', '16872', '16873', '16874', '16875', '16917', '16922', '16923', '16925', '1693', '1694', '17039', '17046', '17048', '17053', '1719', '1729', '1735', '1739', '1740', '1741', '17479', '17481', '17484', '17487', '17488', '17491', '17493', '17501', '17502', '17506', '17507', '17513', '17518', '17529', '17534', '17538', '17540', '17551', '17559', '17562', '17567', '17569', '17573', '17574', '17576', '17579', '17580', '17585', '17589', '17595', '17596', '17597', '1760', '17601', '17612', '17625', '17640', '17643', '17644', '17651', '17652', '17659', '17666', '17671', '17673', '17677', '17682', '17683', '17687', '17688', '17694', '17700', '17701', '17706', '1771', '17710', '17716', '17719', '17723', '17727', '17728', '17738', '17743', '17746', '17749', '17752', '17760', '17762', '17763', '17766', '17770', '17775', '17776', '17777', '17782', '17784', '17785', '17787', '17792', '17798', '17809', '17810', '17812', '17822', '17825', '17827', '17835', '17837', '17839', '17847', '17849', '17853', '17857', '17860', '17863', '17864', '17866', '17867', '1787', '17872', '17873', '17876', '17886', '1789', '1790', '17907', '17911', '17913', '17950', '17956', '17957', '17958', '17960', '17968', '17973', '17978', '1798', '17981', '17984', '17986', '17993', '1801', '18017', '18029', '18036', '18037', '18038', '18040', '18071', '18075', '18079', '1808', '18080', '18101', '18111', '18135', '18138', '18145', '18149', '18157', '18182', '18187', '18188', '18205', '18210', '18223', '18229', '18232', '18236', '18238', '18239', '18252', '18300', '18313', '18317', '18325', '18334', '18336', '18342', '1835', '18351', '1836', '18467', '1847', '18476', '18479', '185', '18504', '18530', '1855', '18566', '18567', '18569', '18570', '18575', '18576', '18577', '18585', '18590', '18592', '186', '1860', '18600', '18605', '1862', '18638', '18643', '18644', '18654', '18658', '18660', '18663', '18667', '18682', '18686', '18688', '18730', '18751', '18778', '18783', '18811', '18842', '18856', '18857', '18859', '1886', '18861', '18863', '18882', '18895', '18898', '18906', '18910', '18921', '18922', '18938', '18959', '18962', '18963', '18975', '18986', '18988', '19006', '19042', '19048', '19050', '19087', '19113', '19118', '19127', '19136', '19137', '19152', '19155', '19159', '19180', '19181', '1920', '19202', '19208', '19240', '19241', '19255', '19256', '19258', '19263', '19270', '19271', '19272', '19280', '19281', '19283', '19286', '19290', '19297', '19307', '19312', '19325', '19327', '19331', '19334', '19341', '19343', '19344', '19351', '19352', '19356', '19359', '19360', '19364', '19368', '19369', '19378', '19379', '19380', '19381', '19383', '19386', '19406', '19409', '19411', '19412', '19419', '19420', '19426', '19428', '19432', '19435', '19441', '19447', '19448', '19451', '19453', '19461', '19462', '19464', '19470', '19473', '19482', '19501', '19507', '19510', '19572', '19630', '19631', '19635', '19645', '19649', '19654', '19663', '19666', '19675', '19679', '19681', '19683', '19690', '19697', '19699', '19702', '19703', '19706', '19711', '19713', '19719', '19726', '19727', '19731', '19732', '19734', '19737', '19744', '19748', '19751', '19758', '19760', '19762', '19763', '19766', '19767', '19769', '19773', '19780', '19796', '19799', '19810', '19823', '19826', '19828', '19830', '19831', '19833', '19837', '19839', '19856', '19860', '19864', '19866', '1987', '19870', '19872', '19874', '19880', '1989', '19890', '19892', '19895', '199', '19900', '19902', '19906', '19912', '19913', '19921', '19930', '19935', '19939', '1994', '19943', '19946', '19950', '19955', '19959', '1996', '19962', '19963', '19965', '19968', '19975', '19981', '19983', '19984', '20035', '20037', '20039', '2010', '20114', '2013', '20137', '20142', '20193', '20210', '20216', '2024', '2027', '20297', '20298', '20306', '20311', '20321', '20328', '20352', '20379', '2039', '20396', '20398', '20439', '20457', '20467', '20485', '20489', '20525', '2054', '20552', '20584', '20618', '20641', '20667', '2067', '20679', '20693', '20698', '20725', '20734', '20744', '20754', '20755', '20766', '20775', '20792', '20797', '2080', '20811', '20824', '20834', '20835', '20840', '20887', '20898', '20901', '20905', '20917', '20921', '20945', '20946', '2097', '20982', '20995', '21000', '21003', '21007', '21019', '2102', '21030', '21039', '21040', '21041', '21046', '21050', '21051', '21059', '21060', '21061', '21063', '21065', '2107', '21071', '21073', '21078', '21082', '21093', '21097', '21129', '21133', '21134', '2114', '21140', '21143', '2118', '21201', '21210', '21228', '21230', '21251', '21253', '2126', '21261', '21278', '2128', '21282', '21287', '21291', '21292', '21297', '21301', '21307', '21318', '21319', '21333', '21357', '21358', '21375', '21376', '21382', '21387', '21393', '21394', '21401', '21404', '21408', '21410', '21411', '21417', '21419', '21421', '21427', '21429', '21440', '21454', '21462', '21466', '21469', '21478', '21503', '2151', '21538', '21550', '21555', '21571', '2159', '2193', '2195', '2202', '2207', '2219', '2220', '2224', '2225', '2237', '2240', '2244', '2247', '2251', '2257', '2268', '2271', '2281', '2291', '2297', '2301', '2322', '2323', '2325', '2344', '2365', '2374', '2375', '2376', '2410', '2419', '2434', '2447', '2462', '2470', '2474', '2475', '2476', '2491', '2497', '2500', '2505', '2512', '2520', '2526', '2528', '2532', '2534', '2538', '2540', '2543', '2546', '2551', '2552', '2564', '2570', '2574', '2577', '2578', '2582', '2583', '2596', '260', '2604', '2609', '2634', '2637', '2638', '2647', '2650', '2652', '2660', '2672', '2699', '2714', '2728', '2737', '2763', '2768', '2769', '2783', '2784', '2791', '2794', '2798', '2799', '2802', '2810', '2811', '2813', '2815', '2818', '2833', '2847', '2867', '2884', '2896', '2897', '2898', '2902', '2907', '2909', '2917', '2924', '2926', '2927', '2933', '2939', '2940', '2945', '2946', '2948', '2950', '2951', '2953', '2956', '298', '2999', '3009', '3013', '302', '3023', '3025', '3035', '3038', '3039', '304', '3041', '3042', '3053', '3056', '3061', '3063', '3069', '3071', '3072', '3074', '3083', '3085', '3086', '3089', '3093', '3094', '3095', '3096', '3100', '3107', '3114', '3120', '3125', '3129', '3139', '3142', '315', '3152', '3153', '3154', '3157', '3166', '3168', '3174', '3176', '3180', '3184', '3186', '3198', '3205', '3216', '3223', '3229', '3235', '3239', '3241', '3242', '3251', '3252', '3261', '3263', '3264', '3266', '3267', '3276', '3277', '3287', '3290', '3295', '3298', '3302', '3305', '3309', '331', '3313', '3320', '3333', '3336', '334', '3345', '3356', '3363', '3367', '3370', '3372', '3377', '3378', '3381', '3384', '3387', '3394', '3395', '3396', '3399', '3400', '3406', '3409', '3413', '3415', '3416', '3417', '3418', '3422', '3425', '3459', '3477', '3491', '3547', '3565', '3570', '3573', '3586', '3587', '3591', '3596', '3599', '3607', '361', '362', '3622', '3632', '3645', '3652', '366', '3673', '3684', '369', '3707', '371', '3711', '372', '3741', '3744', '3747', '3759', '376', '3766', '3775', '3779', '379', '3790', '3795', '3797', '3821', '3828', '383', '3833', '3839', '3856', '3864', '3868', '387', '3873', '3879', '389', '3906', '3910', '3912', '3916', '3918', '3920', '3926', '393', '3933', '3934', '3939', '3948', '3960', '3961', '3964', '3970', '398', '3983', '3984', '3990', '3991', '3993', '3999', '400', '401', '4032', '4033', '4043', '4052', '4055', '4058', '4064', '4069', '408', '4107', '4127', '4135', '4145', '4146', '4148', '4153', '4158', '4159', '4161', '4164', '4167', '418', '4182', '4201', '4204', '4209', '4212', '4227', '4229', '4233', '4235', '4237', '424', '4249', '4277', '4281', '4282', '4284', '4301', '4303', '4310', '4315', '4323', '4339', '4343', '436', '4360', '4367', '4369', '4371', '44', '440', '4401', '4408', '441', '4411', '442', '4434', '4447', '4465', '447', '4477', '448', '4488', '4493', '45', '4510', '4518', '4536', '4537', '4579', '4586', '4591', '4592', '4605', '4606', '4615', '4622', '4623', '4659', '4665', '467', '4690', '473', '4733', '474', '4748', '4758', '4761', '4772', '4774', '478', '4798', '4806', '4807', '4812', '4818', '4841', '4850', '4884', '4887', '4889', '4890', '4893', '4894', '4904', '4908', '4910', '4918', '4933', '4937', '4952', '4955', '4959', '496', '4963', '4964', '497', '4970', '4976', '498', '4981', '5001', '5020', '504', '5044', '505', '5077', '5080', '5081', '5082', '5091', '5094', '5097', '5102', '5112', '5113', '5116', '5117', '5121', '5141', '5162', '5177', '5203', '5212', '5226', '5234', '5272', '5284', '5286', '5287', '5291', '5293', '5299', '5303', '5304', '5309', '532', '5320', '5329', '5331', '5332', '5335', '5340', '5349', '5350', '5352', '5357', '5362', '5373', '5379', '5396', '5398', '5409', '5411', '5435', '5441', '5442', '545', '5455', '5465', '5469', '547', '5477', '550', '5505', '551', '5516', '5517', '5520', '553', '5533', '5559', '558', '559', '5596', '5597', '5608', '5609', '5615', '5634', '5641', '5643', '5652', '5659', '5661', '5664', '5665', '5667', '5670', '5671', '5672', '5675', '5686', '5690', '5693', '5708', '5710', '5713', '5724', '5732', '5739', '5740', '5750', '5751', '5754', '5755', '5756', '5757', '5762', '5780', '5789', '5808', '5854', '5873', '5875', '5876', '5889', '5899', '5900', '5902', '5910', '5915', '5919', '5934', '5937', '5946', '5947', '5949', '5955', '5974', '5975', '5977', '5986', '6003', '6020', '6026', '6027', '6030', '6034', '6056', '6059', '6065', '6068', '6069', '6070', '6082', '6085', '6090', '6092', '6093', '6096', '6101', '6103', '6120', '6121', '6124', '6135', '6157', '6180', '6188', '6199', '6212', '6217', '6223', '6225', '6246', '6276', '6277', '6278', '6282', '6315', '6331', '6337', '6340', '6346', '6361', '6368', '6373', '6382', '6402', '6410', '6422', '6428', '6430', '6433', '6435', '6441', '6442', '6443', '6449', '6463', '6473', '6477', '6488', '6497', '6498', '6499', '6502', '6511', '6527', '6537', '6538', '6556', '6562', '6604', '6616', '6630', '6638', '6639', '6654', '6655', '6659', '6665', '6666', '6675', '6682', '6689', '6694', '6695', '6697', '6702', '6707', '6710', '6719', '6725', '6731', '6767', '6788', '6789', '679', '68', '6805', '6807', '6810', '6811', '6815', '6817', '6818', '6819', '682', '6841', '6844', '6849', '6853', '6855', '6856', '6869', '6873', '6874', '6884', '6886', '6887', '6888', '6908', '6914', '6928', '6929', '6930', '6944', '6953', '6961', '7024', '703', '7030', '7039', '7041', '7045', '7046', '7057', '706', '7065', '7068', '709', '7090', '7093', '7109', '7110', '7111', '712', '7130', '7136', '714', '7149', '7168', '7199', '7218', '7220', '7225', '7234', '724', '7240', '7247', '725', '7253', '7260', '7296', '730', '7305', '7315', '7319', '732', '733', '7337', '7342', '7344', '735', '7352', '7353', '7354', '736', '7362', '7370', '7386', '7391', '7407', '741', '7433', '745', '7453', '7464', '7469', '7470', '7475', '748', '7482', '7484', '7490', '7494', '7497', '7504', '7539', '7548', '7558', '757', '7570', '7593', '7596', '7603', '7604', '7605', '762', '7627', '7635', '7637', '764', '7641', '7644', '7650', '7659', '767', '7670', '7672', '7678', '7679', '768', '7693', '7698', '7704', '7715', '7729', '7733', '7737', '7747', '7749', '7765', '7773', '7784', '7785', '7789', '7791', '7795', '7796', '7810', '7813', '7814', '7816', '7817', '7819', '7820', '7824', '7825', '7831', '7839', '7847', '7855', '7859', '7862', '7865', '7867', '787', '7871', '7891', '7895', '7896', '7897', '7899', '7904', '7912', '7927', '7928', '7930', '7935', '7940', '7959', '7963', '7964', '7966', '7967', '7968', '7969', '797', '7977', '7983', '7984', '7992', '7993', '7997', '8002', '8010', '8020', '8021', '8023', '8040', '8085', '8091', '8100', '8115', '8123', '8126', '8130', '8166', '820', '8204', '8205', '8217', '8220', '8222', '8224', '8225', '8255', '8259', '8264', '8279', '8287', '8288', '8293', '8298', '8327', '8351', '8353', '8355', '8359', '836', '8361', '8367', '8368', '8370', '8373', '8387', '8396', '840', '8406', '8428', '8429', '8430', '8432', '8433', '8438', '844', '8441', '8467', '8472', '849', '8496', '8497', '850', '8509', '851', '8512', '8514', '8518', '853', '8536', '8543', '8544', '8546', '8558', '8561', '8567', '8571', '8572', '8615', '8617', '8618', '8628', '8666', '8685', '8689', '8690', '8691', '8704', '8709', '8711', '8717', '8730', '8734', '8748', '8756', '8758', '8771', '8778', '8780', '8781', '8795', '882', '8822', '8823', '8830', '8836', '8849', '8853', '8854', '8855', '8877', '8880', '8906', '8909', '8916', '8918', '8919', '8932', '8944', '8945', '8947', '8962', '8974', '8975', '8977', '8987', '8996', '8998', '9010', '9020', '9025', '9027', '9033', '9036', '904', '9041', '9045', '9047', '9059', '9064', '9067', '9068', '9075', '9081', '9087', '9088', '9090', '9098', '9102', '9108', '9110', '9112', '9147', '9152', '9156', '9172', '9210', '9230', '9234', '9239', '9243', '9246', '9252', '9258', '9259', '9264', '9277', '9283', '9285', '9288', '9296', '9302', '932', '9324', '9332', '9338', '9343', '9357', '9363', '9365', '9381', '9382', '939', '9392', '9397', '941', '9410', '9412', '9423', '9431', '9432', '9434', '9435', '9443', '9454', '9469', '9472', '9487', '9497', '9499', '9509', '9510', '9526', '9538', '9539', '9544', '9557', '9561', '9565', '9570', '9582', '9597', '96', '960', '9607', '9608', '9619', '9626', '963', '9635', '9637', '9638', '9640', '9642', '9644', '9661', '9668', '9677', '9731', '9733', '9737', '9742', '9755', '9779', '978', '9809', '9814', '982', '9823', '9833', '9844', '985', '9850', '9858', '9867', '9872', '9891', '9893', '9913', '9918', '9920', '9925', '9926', '9928', '9929', '9942', '9947', '9954', '9971', '9976', '9977', '9978', '9984'],
    'alum': ['10258', '10322', '10324', '10593', '10709', '11124', '11273', '11371', '11767', '12484', '1263', '12836', '1300', '1332', '13507', '15368', '16179', '16318', '16320', '16750', '16868', '17698', '17702', '17714', '17870', '17874', '18093', '1832', '18356', '18920', '18930', '19037', '19041', '19329', '19367', '19540', '20055', '20058', '21526', '272', '377', '3957', '4697', '5149', '5243', '5868', '6189', '6546', '7263', '7574', '7577', '7602', '8873', '921', '9326', '938', '9738', '9892'],
    'barley': ['10175', '1067', '11208', '11316', '11885', '12428', '13099', '13744', '13795', '13852', '13856', '15618', '15649', '15676', '15728', '15871', '15875', '15952', '1652', '17767', '17769', '18024', '18263', '18908', '19275', '19668', '1970', '2044', '2171', '2172', '2191', '2217', '2232', '3132', '3324', '395', '4280', '4296', '5', '501', '5467', '5610', '5640', '6626', '7205', '7579', '8213', '8257', '8759', '9865', '9958'],
    'bop': ['10294', '10295', '10297', '10341', '10362', '10372', '10376', '1050', '1100', '11127', '11137', '11185', '11882', '12986', '14109', '14862', '15543', '15573', '15646', '15793', '16108', '1611', '16158', '16226', '16759', '16766', '17916', '17924', '17926', '18347', '18496', '18748', '18753', '1890', '18985', '18991', '18996', '19005', '19061', '19075', '19444', '19568', '1961', '1964', '21096', '21392', '21473', '21519', '21525', '227', '3019', '3024', '3065', '312', '3249', '3338', '3488', '3512', '3541', '3610', '3615', '4045', '4078', '4366', '4629', '5189', '5554', '5764', '5765', '5779', '5785', '5786', '5838', '5842', '5939', '5941', '5981', '6037', '6156', '6406', '6464', '6596', '6637', '6954', '6959', '7047', '7537', '7543', '7633', '7662', '8174', '880', '890', '897', '9143', '9729', '9747', '9749', '9768', '9770', '9784', '9816', '9821', '9836', '9848'],
    'carcass': ['10124', '10334', '10373', '10616', '1110', '11278', '11544', '121', '12257', '12372', '12376', '12737', '13149', '13380', '13382', '1372', '1393', '1398', '1407', '14858', '15254', '1557', '15642', '15798', '16379', '16564', '16784', '1772', '18034', '18099', '18276', '18277', '18352', '18482', '19269', '1970', '20441', '20462', '20511', '21367', '2226', '2389', '2448', '2495', '2978', '3084', '3318', '3901', '4022', '4101', '4503', '4514', '4577', '4631', '4636', '5009', '5031', '5445', '5487', '5812', '5896', '6269', '6317', '6804', '8574', '8800', '9060', '9754'],
    'castor-oil': ['10300', '19672'],
    'cocoa': ['1', '10014', '10122', '10403', '10449', '10471', '10491', '10505', '10506', '10584', '10586', '10613', '10619', '10742', '10760', '11224', '11341', '11459', '11462', '11811', '11843', '12401', '12763', '12813', '13271', '13462', '15095', '15179', '15653', '16098', '16148', '17499', '17568', '17664', '17707', '17733', '17984', '18014', '18221', '18222', '19358', '19500', '19570', '20005', '2521', '275', '3190', '3225', '3310', '4147', '4470', '5168', '5192', '5258', '5382', '5491', '5598', '5880', '6128', '6405', '6407', '6414', '6493', '7071', '7311', '8326', '8850', '8961', '8978', '9450', '9559', '9903', '9953'],
    'coconut': ['10712', '10720', '11836', '17477', '20045', '8112'],
    'coconut-oil': ['13080', '17477', '18781', '20911', '3540', '6421', '8596'],
    'coffee': ['10014', '10100', '1026', '10268', '1030', '10406', '10640', '10682', '10752', '1085', '10876', '10902', '10959', '11183', '11224', '11265', '11341', '11372', '11462', '11816', '11866', '11882', '11949', '12008', '12011', '1207', '1212', '12152', '12208', '12340', '12355', '12399', '12424', '12426', '1246', '12465', '12490', '12814', '12843', '1312', '13170', '13190', '13201', '13242', '13269', '13834', '14418', '14840', '15198', '15329', '15357', '15540', '15584', '15725', '15737', '1579', '16097', '16115', '16279', '17038', '1715', '17664', '18323', '1842', '18604', '1880', '18860', '1889', '1910', '19318', '19387', '19458', '19480', '19570', '1960', '19672', '19807', '20186', '20382', '20465', '2115', '21177', '21567', '232', '235', '2467', '249', '2521', '2550', '2553', '2606', '275', '290', '2954', '3034', '3040', '3187', '3310', '3559', '3955', '402', '4063', '4071', '4147', '42', '4267', '4349', '4603', '4785', '5002', '5134', '5334', '5471', '5570', '562', '6595', '6632', '6912', '7104', '7124', '7143', '7367', '7406', '75', '754', '7888', '8105', '8149', '8193', '8200', '842', '875', '8903', '8950', '9153', '9265', '9654', '9680', '977'],
    'copper': ['1148', '1184', '11945', '12024', '12215', '12223', '12484', '12489', '12857', '12861', '12910', '12980', '12992', '13003', '13694', '13877', '14297', '14476', '14499', '14572', '14805', '14852', '15264', '15831', '15932', '15988', '1607', '16110', '16123', '17509', '17714', '18225', '18280', '18317', '18339', '18849', '18952', '19781', '19786', '2006', '20096', '2074', '2186', '22', '2764', '2782', '2880', '3454', '3613', '3862', '4291', '4431', '5435', '5788', '5827', '5888', '6025', '6846', '6927', '7552', '7775', '793', '800', '816', '9195'],
    'copra-cake': ['17477', '235', '267'],
    'corn': ['10120', '10139', '10172', '10175', '10319', '10339', '10487', '10489', '10519', '10701', '10882', '10956', '11012', '11085', '11091', '11269', '1131', '11392', '11436', '11607', '11612', '11729', '11739', '11769', '11885', '11936', '11939', '11964', '12002', '12052', '12055', '1215', '12160', '12311', '12323', '12372', '12417', '12436', '12500', '12583', '12587', '1268', '1273', '12872', '13173', '13179', '1369', '1385', '13852', '13856', '1395', '1399', '14483', '14832', '14858', '15033', '15043', '15106', '15287', '15341', '15618', '15648', '15676', '15686', '15720', '1582', '15845', '15856', '15860', '15863', '15871', '15875', '15877', '15890', '15904', '15906', '15910', '15911', '15917', '15952', '15999', '16012', '16071', '16099', '16147', '1652', '16525', '16624', '16751', '16765', '17503', '17509', '17722', '1777', '18035', '1843', '18482', '18614', '18954', '18973', '19165', '193', '1952', '197', '19721', '19821', '20018', '20366', '2044', '20637', '20645', '20649', '20723', '20763', '21091', '21243', '21493', '2172', '2183', '2217', '2264', '235', '2382', '2436', '2456', '2595', '2599', '2617', '2727', '2741', '2749', '2777', '2848', '2913', '2922', '2947', '3138', '3191', '327', '3282', '3299', '3306', '3330', '3337', '3358', '3401', '3429', '3847', '3855', '3881', '3949', '395', '3979', '3981', '4047', '4133', '4289', '4296', '4382', '4490', '4599', '4825', '4905', '4939', '4988', '5', '5003', '501', '5017', '5033', '5109', '516', '5185', '5338', '5467', '5518', '5531', '5606', '5610', '5636', '5637', '57', '5847', '5933', '6', '6142', '6221', '6236', '6239', '6259', '6269', '6386', '6585', '6588', '6735', '6890', '6897', '694', '7062', '7205', '7215', '7336', '7387', '7389', '7390', '7395', '7700', '7792', '7917', '7934', '7943', '8004', '8140', '8161', '8166', '8257', '8273', '8400', '8443', '8446', '8535', '855', '8759', '8941', '8983', '8993', '9058', '9093', '9094', '934', '9470', '9521', '9667', '97', '9865', '9958', '9989'],
    'cotton': ['10062', '104', '10537', '11064', '11091', '11507', '11609', '11640', '11671', '11769', '11970', '12254', '12372', '12969', '1405', '14354', '15220', '15568', '15643', '15853', '15871', '15875', '15876', '15890', '1590', '15906', '15925', '15973', '16009', '16107', '1882', '18873', '19570', '19672', '19835', '19947', '20019', '21530', '235', '2359', '2456', '2975', '374', '4056', '4103', '4231', '4549', '4679', '5040', '5467', '5610', '6063', '6269', '6535', '7640', '7917', '8273', '8413', '8759'],
    'cotton-oil': ['15871', '18781', '6412'],
    'cpi': ['10260', '10261', '10377', '10384', '10385', '10553', '10567', '10724', '11197', '1147', '11894', '12080', '12464', '12771', '13007', '13172', '13333', '14770', '14891', '14918', '1550', '16103', '16108', '16118', '16134', '16158', '16226', '16241', '16268', '16415', '16740', '16783', '1681', '17442', '179', '17970', '18001', '18987', '18993', '19000', '19005', '19079', '19081', '19513', '19625', '19653', '20247', '209', '21130', '213', '21532', '2529', '2555', '3024', '3411', '3593', '4486', '4548', '4555', '4573', '4741', '5145', '5194', '5231', '5238', '5245', '5260', '5318', '5375', '5378', '5389', '5490', '5523', '5830', '6385', '6406', '6920', '6965', '6982', '7177', '7314', '7580', '7584', '7590', '7591', '8119', '8164', '8211', '8694', '8726', '8746', '9165', '929', '9294', '9433', '9824', '9829'],
    'cpu': ['21245', '5388', '5460', '5485'],
    'crude': ['10011', '10078', '10080', '10106', '10168', '10190', '10192', '10200', '10228', '1026', '10268', '10395', '10406', '10452', '10539', '10567', '10588', '10620', '10621', '10627', '10632', '10641', '10669', '10693', '10750', '10797', '10845', '10873', '10947', '11007', '11025', '11100', '11118', '11149', '11213', '11231', '11350', '11388', '11403', '11421', '11444', '11455', '11491', '11559', '11632', '11639', '11699', '11723', '11731', '12267', '12286', '12371', '12503', '12533', '127', '12775', '12803', '12940', '1306', '13096', '13102', '13115', '13142', '13184', '13200', '1324', '13256', '13266', '1331', '1335', '1343', '13542', '13611', '13633', '1387', '13963', '14211', '14395', '144', '14698', '14704', '14709', '14732', '14829', '15063', '15200', '1521', '15230', '15238', '15244', '15322', '15339', '15344', '15351', '15520', '1556', '15939', '15964', '16005', '16007', '16040', '16077', '1616', '16366', '16429', '16438', '16593', '16607', '1661', '16636', '16649', '16651', '16658', '16710', '16723', '16739', '16762', '1686', '1692', '17054', '1709', '1711', '17478', '17516', '17519', '17584', '17618', '17658', '17669', '17750', '17757', '17771', '17780', '17813', '17816', '17875', '17886', '17888', '17892', '17913', '17929', '17963', '1799', '18066', '18085', '18108', '18146', '18186', '18213', '18234', '18311', '18325', '18329', '18332', '18340', '18493', '1851', '18521', '18523', '1856', '18621', '18651', '18655', '18678', '18680', '18689', '18692', '18698', '18701', '18704', '18705', '18706', '18707', '18728', '18736', '18738', '18743', '18746', '18747', '1875', '18754', '18765', '18773', '18774', '1878', '18789', '18795', '18810', '18824', '18840', '18857', '18896', '189', '1904', '19069', '1909', '191', '19110', '19128', '19182', '19193', '19285', '19291', '19397', '194', '19403', '1948', '19492', '19497', '19499', '19505', '19506', '19509', '19556', '19559', '19560', '19684', '19756', '1980', '19832', '19844', '19869', '1990', '19903', '19927', '19996', '19998', '20008', '20030', '2004', '2007', '20090', '20092', '20093', '20095', '20101', '20103', '20270', '20333', '20420', '20459', '2046', '20464', '20474', '2061', '20632', '20653', '20662', '20692', '20709', '20721', '20730', '20756', '20774', '20778', '20828', '20869', '20878', '20881', '20882', '20890', '20909', '20919', '20936', '20944', '20959', '20981', '20991', '21002', '21006', '21013', '21018', '21058', '21067', '21076', '211', '21131', '21149', '21197', '2121', '21216', '21267', '21274', '21363', '21369', '21380', '21417', '21443', '21459', '21465', '21475', '21482', '21484', '21485', '21492', '21502', '21506', '21541', '21568', '2174', '2175', '2187', '2231', '236', '237', '2383', '242', '2449', '246', '248', '2511', '2515', '2517', '2522', '2530', '2585', '2688', '273', '2767', '2775', '2838', '2925', '2957', '2970', '2973', '2998', '3003', '3015', '3017', '3048', '3065', '3115', '3145', '3146', '3162', '3169', '3174', '3189', '3204', '3303', '3332', '3342', '3354', '3364', '3389', '3430', '3452', '3455', '349', '3507', '3509', '352', '353', '3556', '3563', '3571', '3592', '3594', '3609', '368', '3798', '3843', '3869', '3906', '3959', '3976', '3980', '3985', '3995', '4016', '4028', '4039', '4041', '4125', '4129', '4162', '4171', '4174', '4232', '4246', '4315', '4333', '4340', '4365', '4367', '4386', '4425', '4429', '4445', '4453', '4462', '4466', '4467', '4474', '4481', '4525', '4558', '4569', '4576', '4578', '4584', '459', '4590', '4593', '4600', '4604', '4609', '4658', '4662', '4664', '4689', '4713', '4714', '4742', '4831', '4848', '4867', '489', '4951', '4953', '4962', '4983', '502', '5037', '5061', '511', '5116', '5118', '5119', '5123', '5125', '5150', '5156', '5166', '5167', '5171', '5244', '5268', '5270', '5273', '5281', '543', '5542', '5553', '5630', '5683', '5796', '5852', '5866', '6023', '6060', '6086', '6111', '6119', '6125', '6159', '6163', '6166', '6169', '6177', '6184', '6201', '6208', '6264', '6294', '6301', '6348', '6404', '6413', '6432', '6578', '6598', '6606', '6652', '6656', '6722', '6742', '6746', '6760', '6791', '6871', '6876', '6893', '6905', '6913', '6996', '704', '7067', '708', '7097', '7150', '7152', '7174', '7287', '7355', '7423', '7496', '7528', '7529', '7611', '7618', '7639', '7684', '7742', '7854', '791', '8015', '8041', '8069', '8089', '8100', '8117', '8131', '8134', '8160', '8167', '8188', '8209', '8210', '8288', '835', '8402', '8405', '8421', '8440', '8478', '8493', '8516', '8553', '8598', '8600', '8606', '8610', '8623', '8630', '8672', '8675', '873', '8749', '8755', '8765', '8812', '8815', '8820', '8835', '8856', '8882', '8914', '8959', '8964', '8971', '9031', '9039', '9065', '9155', '918', '9208', '9253', '9279', '9293', '930', '9436', '9445', '945', '952', '9527', '9583', '9614', '9634', '9639', '9650', '9674', '9718', '9801', '9849', '988', '9913', '9947'],
    'dfl': ['10344', '17620', '3024'],
    'dlr': ['10075', '10081', '10263', '10308', '10323', '10337', '10357', '10364', '10370', '10382', '10475', '10617', '10650', '10651', '10652', '10653', '10654', '10657', '10658', '10659', '10660', '10679', '10681', '10684', '10689', '10696', '10731', '10736', '10751', '10762', '10766', '10769', '10770', '10782', '11120', '11135', '11162', '11163', '11164', '11212', '11254', '11764', '11772', '11774', '11783', '11844', '11944', '11972', '12136', '12145', '12337', '12396', '12422', '12437', '12470', '12522', '12780', '12795', '12877', '13315', '13512', '13524', '13531', '13544', '13660', '13831', '13869', '13903', '13919', '13988', '14767', '14770', '14777', '14779', '14913', '14931', '1514', '15212', '1533', '15431', '15442', '15449', '15460', '15549', '15562', '15625', '15677', '15689', '16053', '16066', '16067', '16068', '16069', '16106', '16111', '16189', '16200', '16743', '16744', '17041', '17044', '17045', '17047', '17871', '18010', '18012', '18142', '18363', '18370', '19021', '19023', '19028', '19061', '1926', '20862', '20868', '21157', '21172', '21496', '21542', '21573', '2181', '2190', '2327', '2354', '2452', '2623', '2633', '3019', '3020', '3065', '3076', '3267', '3338', '3419', '3421', '425', '4616', '4675', '4882', '5290', '5778', '6108', '8097', '8189', '8584', '8590', '8591', '8621', '8657', '8664', '8714', '872', '8735', '9022', '9054', '9061', '9079', '9118', '9120', '9128', '9129', '9162', '9166', '9213', '9214', '9222', '9224', '9282', '9295', '9377', '9783', '9862', '9871', '9946'],
    'dmk': ['10718', '10751', '11212', '11944', '11972', '12522', '12525', '12877', '13919', '17620', '18136', '21496', '21542', '3076'],
    'earn': ['10000', '10002', '10008', '10023', '10027', '1003', '10032', '10035', '10037', '10040', '10041', '10046', '10050', '10053', '10054', '10057', '10061', '10064', '10067', '10068', '1007', '10073', '10074', '10076', '10083', '10085', '10089', '1009', '10090', '10092', '10094', '10095', '10096', '10097', '10099', '1010', '10105', '10108', '10109', '1011', '10110', '10112', '10114', '10115', '10119', '1013', '10130', '10140', '1015', '1016', '10162', '10163', '10165', '1017', '10173', '10174', '10180', '10181', '10185', '10186', '1019', '10196', '10197', '10201', '10207', '10208', '10211', '10215', '10218', '10222', '10225', '10227', '10231', '10233', '10235', '10237', '1024', '10240', '10242', '10243', '10245', '10253', '10254', '10272', '10280', '10281', '10282', '10284', '10291', '10292', '10318', '10328', '1033', '10338', '10342', '10349', '10381', '10387', '10390', '10393', '10398', '10401', '10402', '10411', '10412', '10427', '10430', '10431', '10432', '10436', '10439', '1044', '10440', '10445', '10447', '10459', '1047', '10470', '10472', '10478', '10484', '10498', '10499', '10500', '10509', '10515', '10524', '10529', '10532', '10534', '10536', '10538', '10542', '10544', '10547', '10548', '10556', '10561', '10564', '10565', '10566', '10573', '1059', '10597', '10598', '10599', '10600', '10601', '10609', '10610', '10611', '10612', '1063', '10644', '10648', '1065', '1066', '10685', '10688', '10699', '1070', '10702', '10703', '1071', '10715', '10738', '10743', '10790', '108', '1080', '10805', '10817', '10822', '10826', '10832', '10833', '10839', '1084', '10840', '10841', '10847', '10851', '10853', '10855', '10856', '1087', '10870', '10877', '10879', '10884', '10885', '10891', '10893', '10894', '10897', '1090', '10904', '10907', '1091', '10917', '10919', '10921', '10922', '10923', '10924', '10925', '10926', '10943', '10944', '10946', '1096', '10963', '10964', '10966', '10972', '10973', '10974', '10980', '10983', '10999', '11', '1101', '11015', '11018', '11019', '11027', '11028', '11031', '11034', '11035', '11036', '11046', '11047', '11049', '1105', '11057', '11058', '11060', '1108', '11087', '1109', '11090', '11098', '11105', '11106', '11109', '1111', '11111', '11112', '11115', '11123', '11124', '11128', '11132', '11145', '11153', '11154', '11156', '11158', '11168', '1117', '11171', '11181', '11196', '11205', '11211', '1122', '11220', '11221', '11227', '11232', '1124', '11241', '11243', '11249', '11270', '11272', '11277', '1128', '11282', '11283', '113', '11300', '11301', '11302', '11305', '11306', '11307', '11308', '11317', '11320', '11333', '11335', '11336', '11338', '1134', '11340', '1135', '11354', '11358', '11359', '1137', '11370', '11377', '1138', '11381', '11388', '11394', '11395', '11400', '11407', '11413', '11416', '11418', '11424', '11428', '11429', '1143', '11433', '11435', '11440', '11443', '11451', '11452', '11470', '11475', '11476', '11478', '11479', '11481', '11490', '11494', '11497', '11499', '11500', '11501', '11503', '11506', '11510', '1152', '11528', '11529', '11540', '11543', '11549', '11564', '11565', '11566', '11567', '11568', '11569', '1157', '11570', '11571', '11572', '11575', '11578', '11588', '1160', '11614', '11617', '11621', '11622', '11628', '11629', '11630', '11633', '11635', '11636', '11637', '11643', '11644', '11649', '11663', '11664', '11666', '11667', '11668', '1167', '11674', '11682', '11683', '11684', '11685', '11687', '11689', '1169', '11690', '11696', '11697', '1170', '11700', '11701', '11702', '11705', '11708', '11711', '11713', '11714', '11717', '11721', '11724', '11725', '11726', '11727', '11728', '1173', '11730', '11733', '11736', '11750', '11752', '11754', '11756', '11758', '1176', '1177', '11773', '1178', '11787', '11802', '1181', '1182', '11850', '11855', '11856', '11859', '11877', '11878', '11879', '1188', '11883', '11884', '11896', '11898', '11904', '11908', '1191', '11918', '1192', '11925', '1193', '11930', '11933', '11947', '11966', '11967', '11976', '11977', '11978', '11979', '1198', '11980', '11983', '11990', '11993', '11994', '12', '1200', '12001', '12005', '12014', '12017', '12022', '12031', '12033', '12035', '12036', '12042', '12044', '12045', '12048', '1205', '12050', '12059', '12061', '12063', '12064', '12067', '12077', '12078', '12079', '1208', '12085', '12098', '12110', '12114', '12115', '12119', '12128', '12149', '12150', '12153', '12154', '12155', '12159', '12163', '12166', '12169', '12176', '12182', '12184', '12185', '12187', '12201', '12202', '12204', '12207', '12211', '12216', '12227', '12231', '12237', '12239', '12240', '12244', '1225', '12255', '12258', '12266', '12271', '12283', '12285', '12288', '1229', '12293', '12294', '12296', '123', '12301', '12303', '12304', '12306', '12317', '12318', '12334', '1234', '12341', '12346', '12351', '12352', '12353', '12362', '12364', '12379', '12380', '12382', '12383', '12389', '1239', '12391', '12394', '12397', '12400', '12408', '12421', '12432', '1244', '12448', '1245', '12463', '12487', '12488', '12505', '12506', '12508', '1251', '12511', '12515', '12516', '12517', '1252', '12520', '12521', '12524', '12531', '12545', '12549', '12550', '12551', '12559', '12562', '12569', '12570', '12572', '12573', '12574', '12579', '1258', '12580', '126', '1260', '1264', '1265', '1269', '1271', '1277', '12783', '12799', '12824', '1283', '12831', '1284', '1285', '12860', '12862', '12869', '1287', '12874', '12878', '1288', '12885', '12886', '12893', '12897', '12898', '129', '12901', '12904', '12914', '12915', '12917', '12929', '1293', '12949', '12962', '12978', '1298', '13', '130', '13006', '1301', '13014', '13025', '13026', '13027', '13031', '13032', '13059', '13062', '13064', '13078', '13092', '13093', '1311', '13110', '13113', '13114', '13118', '13122', '13126', '1313', '13131', '13136', '13138', '1314', '13141', '13148', '1315', '13165', '1317', '13174', '13180', '13186', '13187', '13191', '13192', '1320', '13203', '13204', '13214', '1322', '13223', '13224', '13225', '1326', '13263', '1327', '1328', '13336', '1339', '1341', '1342', '13435', '1344', '1345', '1351', '1355', '1359', '1360', '13615', '1363', '1364', '13656', '1366', '1367', '13695', '13708', '13709', '13710', '13715', '13717', '13721', '13735', '1375', '1378', '1379', '138', '1381', '1382', '1384', '13854', '13890', '139', '13900', '13904', '1391', '13966', '14', '140', '1404', '14069', '14091', '14093', '14103', '1414', '14166', '14168', '14180', '14182', '142', '1420', '1428', '14282', '14285', '143', '14300', '14310', '14314', '14316', '14324', '1433', '1434', '1435', '1438', '14388', '14393', '14402', '14417', '14427', '1445', '1446', '1449', '145', '14516', '14517', '1452', '14524', '14525', '14539', '14540', '14544', '14545', '14547', '14549', '14550', '14551', '1456', '14569', '14586', '146', '1460', '1461', '14622', '14629', '14630', '14646', '1466', '14666', '1467', '14672', '1469', '14697', '147', '1471', '14729', '14744', '14751', '14752', '14755', '14761', '1483', '14859', '14860', '14872', '14873', '14875', '14876', '1488', '14899', '14903', '14911', '1492', '14926', '14930', '14933', '14934', '1494', '14954', '14958', '1496', '14960', '14962', '14963', '14965', '14967', '14968', '14969', '14970', '14971', '14975', '1498', '14981', '14983', '14984', '14985', '14993', '14995', '14998', '15000', '15002', '15005', '15006', '15011', '15012', '15016', '1502', '15020', '15023', '15026', '15027', '15028', '15029', '15032', '15046', '15053', '15055', '15056', '15062', '15063', '15065', '15067', '15070', '15074', '15077', '15078', '15079', '1508', '15082', '15090', '15093', '15094', '151', '15103', '15104', '15107', '15109', '15110', '15118', '15120', '15121', '15126', '15130', '15136', '15141', '15144', '15145', '15152', '15153', '15157', '15162', '15172', '15175', '15180', '15185', '15188', '15189', '15190', '15193', '15194', '15197', '152', '15205', '15207', '15208', '15211', '15213', '15221', '15222', '15226', '15227', '15233', '15238', '1524', '15240', '15243', '15247', '1525', '15250', '15258', '15259', '1526', '15263', '15265', '1527', '15270', '15274', '15278', '1528', '15280', '15283', '1529', '15292', '15295', '15296', '15299', '15300', '15307', '15309', '15312', '15314', '15315', '1532', '15321', '15324', '15325', '15326', '15327', '15336', '15345', '15348', '15354', '15356', '15359', '1536', '15379', '15380', '15383', '15387', '15389', '15391', '15396', '15397', '15406', '1541', '15413', '15417', '15427', '15429', '1543', '15438', '1545', '15451', '15457', '15459', '1546', '15476', '15477', '15478', '15482', '15489', '15494', '15495', '15496', '15511', '15515', '15521', '15528', '15535', '1554', '15547', '15548', '1555', '15553', '15558', '15559', '15566', '15569', '15570', '15581', '15583', '15585', '15590', '15593', '15594', '15597', '15598', '15601', '15602', '15613', '15626', '1563', '15632', '15634', '15637', '1564', '15640', '15641', '15651', '15655', '1566', '15667', '15668', '15669', '1567', '15674', '15679', '1568', '15680', '15688', '1569', '15691', '15692', '15695', '15702', '15703', '15704', '15707', '15708', '15709', '1571', '15715', '15719', '1572', '15723', '15729', '15732', '15733', '15736', '15749', '1575', '15753', '15757', '15759', '15762', '15769', '15772', '15777', '15780', '15782', '15790', '15797', '15800', '15801', '15807', '15808', '15819', '15821', '15822', '15823', '15832', '15833', '15834', '15841', '1585', '15851', '15852', '1586', '15867', '15878', '15881', '15886', '15888', '15889', '15896', '15897', '15899', '15901', '15902', '15908', '15909', '15913', '15930', '15933', '15934', '15937', '15944', '15959', '15967', '15969', '15979', '15980', '15981', '15984', '15985', '160', '16003', '16013', '16015', '16016', '1602', '16022', '16023', '16030', '16045', '16055', '1608', '16086', '1610', '16112', '16119', '16140', '16141', '16170', '16172', '16175', '1620', '16202', '16206', '16207', '16219', '16221', '16228', '16230', '16234', '16244', '16246', '16252', '16260', '16262', '16263', '16264', '16266', '16269', '16275', '16278', '16283', '16284', '16288', '16291', '16294', '16297', '16299', '163', '16300', '16301', '16303', '16311', '16312', '16315', '16317', '16319', '16324', '1633', '16336', '16337', '16339', '16342', '16343', '16346', '16347', '16348', '16350', '16354', '16360', '16363', '16365', '16367', '16369', '16371', '16372', '16374', '16376', '16377', '1638', '16380', '16385', '16386', '16388', '1639', '16390', '16392', '16393', '16394', '16396', '16399', '164', '16400', '16401', '16402', '16405', '16410', '16417', '16418', '1642', '16421', '16422', '16424', '16426', '16428', '1643', '16430', '16432', '16434', '16437', '1644', '16440', '16441', '16448', '16454', '16458', '16461', '16468', '16469', '16470', '16471', '16472', '16473', '16476', '16478', '1648', '16480', '16481', '16483', '16486', '16487', '16488', '16490', '16492', '16495', '16496', '165', '1650', '16502', '1651', '16518', '16522', '16527', '1653', '16530', '16533', '16538', '16539', '16545', '16546', '16549', '1655', '16551', '16554', '16561', '16568', '16569', '1657', '16574', '1658', '16584', '16585', '16587', '16588', '16590', '16592', '16598', '16599', '166', '16600', '16602', '16606', '16608', '16610', '16611', '16615', '16619', '16623', '16625', '16626', '16628', '16633', '16635', '16638', '16641', '16642', '16644', '16645', '16653', '16654', '16655', '16662', '16664', '16665', '16668', '16670', '16672', '16678', '16679', '16687', '16698', '167', '16705', '16708', '16711', '16715', '16717', '16719', '16725', '16726', '16735', '1677', '16792', '16795', '16797', '16798', '16799', '168', '16800', '16801', '16802', '16803', '16804', '16806', '16807', '16808', '16809', '16810', '16812', '16813', '16814', '16815', '16816', '16817', '16818', '16819', '1682', '16824', '16825', '16826', '16834', '16835', '16836', '16837', '16838', '16839', '16848', '16850', '16858', '16865', '1688', '16884', '16885', '16886', '16888', '1689', '16890', '16893', '169', '16909', '16910', '16911', '16912', '16913', '16916', '16918', '16921', '1696', '17031', '17032', '17034', '17035', '17037', '1706', '1708', '1714', '1716', '1718', '1722', '1727', '1732', '1734', '17440', '1747', '17489', '1751', '1752', '17543', '17544', '17545', '17547', '17554', '17556', '17558', '1758', '1759', '17607', '17616', '17617', '17619', '1763', '17630', '1764', '1765', '17655', '17660', '17661', '1767', '17672', '1768', '17685', '1769', '17703', '17712', '17751', '17753', '17773', '17788', '17791', '17795', '17802', '17808', '17811', '1783', '17834', '17848', '17887', '17900', '17901', '17922', '1794', '17964', '17965', '17967', '1797', '17972', '17975', '17988', '18', '18028', '1803', '18048', '18052', '18062', '18065', '18068', '18096', '18120', '1814', '18152', '18155', '18158', '1816', '18166', '18180', '1820', '1824', '18242', '18243', '18249', '18258', '18283', '18294', '183', '18303', '18307', '18308', '18309', '18326', '18354', '18355', '1837', '1846', '18481', '18483', '1849', '18495', '18497', '18514', '1853', '18563', '18591', '1860', '18602', '18606', '1861', '18610', '18627', '18646', '18648', '18699', '187', '18703', '18708', '18713', '18718', '18719', '1872', '18721', '1873', '18732', '18769', '1877', '18779', '18816', '18828', '18830', '18832', '18870', '18875', '18953', '18955', '18997', '19009', '19029', '19045', '19053', '19142', '1917', '19170', '19171', '19174', '19178', '1919', '1924', '19254', '19267', '19304', '19308', '19357', '19365', '19374', '19375', '19410', '19422', '19424', '19427', '19440', '19450', '19459', '19466', '19474', '19475', '19515', '19561', '19562', '19571', '196', '19639', '19642', '19644', '19647', '19648', '19650', '19651', '19655', '19657', '19667', '19674', '19680', '19704', '19710', '19712', '19723', '19742', '19745', '19761', '19784', '19787', '19788', '19790', '19803', '19812', '19815', '19850', '19854', '19862', '19865', '19887', '19915', '19933', '19957', '19966', '19978', '2001', '2005', '201', '20105', '20112', '20117', '20118', '20119', '20129', '20132', '20135', '20138', '20140', '20151', '2016', '20161', '20165', '20166', '20168', '20170', '20174', '20176', '20180', '20181', '20187', '20189', '20191', '20195', '202', '2020', '20206', '20207', '20213', '20214', '20215', '20217', '20220', '20221', '20223', '20226', '20228', '20233', '20234', '20235', '20237', '20241', '20249', '20251', '20252', '20255', '20256', '20257', '2026', '20266', '20269', '20273', '20274', '20278', '20279', '2028', '20284', '20286', '2029', '20291', '20299', '20300', '20302', '20304', '20308', '20309', '20310', '20314', '20319', '20324', '2033', '20332', '20334', '20337', '20339', '20340', '20344', '2035', '20353', '20357', '20358', '20363', '20364', '20365', '20367', '20368', '20370', '20371', '20372', '20373', '20374', '20376', '20378', '20380', '20381', '20384', '20385', '20386', '20388', '20391', '20392', '20393', '20395', '20400', '20401', '20407', '20412', '20414', '20415', '20419', '2042', '20421', '20432', '20438', '20440', '20442', '20443', '20446', '20448', '20450', '20453', '20455', '20463', '20468', '2047', '20470', '20471', '20473', '20476', '20479', '2048', '20480', '20488', '2049', '20490', '20493', '20494', '20497', '20498', '20499', '20502', '20507', '20514', '20515', '20517', '20523', '20527', '20529', '20531', '20533', '20535', '20545', '20546', '20548', '2055', '20557', '20558', '2056', '20561', '20562', '20564', '20566', '20567', '20569', '20583', '20586', '20593', '20596', '20598', '20600', '20605', '20606', '2061', '20628', '20630', '20634', '20639', '20640', '20646', '20648', '2065', '20650', '20651', '20652', '20654', '20655', '20657', '20658', '20660', '20668', '20669', '20670', '20671', '20672', '20673', '20680', '20685', '20695', '20706', '20708', '20711', '20712', '20715', '20728', '20735', '20736', '20746', '20747', '20753', '20767', '20768', '20770', '20771', '20773', '20776', '20779', '20784', '2079', '20791', '20793', '20794', '20798', '20800', '20801', '20802', '20803', '20804', '20807', '20812', '20813', '20814', '20817', '20822', '20825', '20826', '20833', '20836', '20837', '20838', '20879', '20883', '20885', '20888', '2089', '20894', '20896', '20900', '20904', '20910', '20913', '20915', '20918', '2092', '20920', '20926', '20927', '20930', '20931', '20932', '20937', '20938', '20943', '20947', '20949', '20954', '20968', '20969', '20972', '20973', '20974', '20975', '20976', '20979', '20983', '20984', '20988', '2099', '210', '2101', '21012', '21014', '21016', '21017', '21022', '21028', '21032', '21034', '21036', '21037', '21038', '21042', '21044', '21045', '21047', '21049', '21053', '21055', '21077', '2108', '21081', '2109', '21095', '21100', '21103', '21105', '21106', '21109', '21112', '21113', '21114', '21117', '2112', '21132', '21135', '21136', '21142', '21146', '21148', '2116', '21164', '21165', '21166', '21173', '21175', '21176', '21178', '21179', '21181', '21182', '21189', '21190', '21191', '21192', '21193', '21194', '21195', '21196', '21199', '212', '21203', '21205', '21206', '21208', '21209', '21211', '21212', '21214', '21215', '21218', '21219', '2122', '21221', '21222', '21224', '21227', '21232', '21233', '21234', '21236', '21237', '21238', '21240', '21242', '21244', '21248', '21252', '21255', '21256', '21257', '21259', '21260', '21262', '21264', '21265', '21266', '21271', '21272', '21275', '21281', '21283', '21286', '21295', '21298', '2130', '21300', '21302', '21308', '21315', '21322', '21324', '21325', '2133', '21330', '21336', '21337', '21346', '21350', '21352', '21354', '21356', '21359', '2136', '21360', '21362', '2137', '21372', '21373', '21378', '21385', '21386', '21388', '214', '21403', '21405', '21406', '21412', '21413', '21414', '21415', '21416', '21430', '21431', '21435', '21436', '21437', '21441', '21447', '21448', '21451', '21453', '21455', '21460', '21474', '21479', '2148', '21480', '21488', '21489', '2149', '21490', '2153', '2154', '21551', '21553', '2157', '2163', '2166', '2167', '2173', '2176', '2182', '2188', '2193', '2194', '2196', '2198', '2203', '2205', '2211', '2212', '2213', '2216', '2221', '2227', '2230', '2234', '2236', '2238', '2249', '2252', '2253', '2259', '2260', '2265', '2266', '2270', '2273', '2275', '2276', '2279', '2290', '2298', '23', '2303', '2304', '2307', '2308', '2309', '2312', '2316', '2320', '2332', '2333', '2335', '2337', '2338', '2339', '2340', '2342', '2346', '2347', '2348', '2350', '2353', '2357', '2366', '2370', '2377', '2379', '2380', '2385', '2386', '2391', '2393', '2396', '2397', '2398', '24', '2402', '2405', '2409', '2421', '2428', '2429', '2433', '2435', '2437', '2440', '2458', '2459', '2460', '2471', '2478', '2479', '2482', '2483', '2484', '2488', '2490', '2492', '2494', '2496', '2504', '2513', '2518', '2519', '2539', '2544', '2548', '2549', '2558', '2560', '2561', '2562', '2566', '2567', '2568', '2569', '2579', '2580', '2584', '2587', '2590', '2591', '2593', '2602', '2608', '2611', '2613', '2619', '2630', '2645', '2649', '2658', '2659', '2663', '2664', '2666', '2667', '2671', '2674', '2677', '2687', '2691', '2693', '2695', '27', '2700', '2708', '2711', '2718', '2720', '2722', '2723', '2729', '2742', '2748', '2750', '2751', '2752', '2754', '2756', '2757', '2761', '2771', '2772', '2773', '2779', '2781', '2787', '2789', '279', '2792', '2797', '2800', '2801', '2803', '2806', '2807', '2809', '2820', '2825', '2826', '2827', '2834', '2836', '2839', '2849', '2850', '2856', '2866', '2873', '2877', '2886', '2887', '2890', '2908', '2912', '2915', '2916', '2918', '2919', '2920', '2928', '2929', '2930', '2938', '2960', '2961', '2962', '2963', '299', '3008', '3026', '3037', '3062', '3077', '3080', '3091', '3117', '3118', '3161', '317', '3194', '3242', '3243', '3247', '3254', '3258', '3269', '3271', '3273', '3275', '3343', '3344', '3359', '3380', '3382', '3405', '3420', '345', '3456', '3463', '3466', '3468', '3474', '3476', '3480', '3481', '3486', '3490', '3497', '3505', '3517', '3538', '3543', '3545', '3557', '356', '3564', '3569', '3575', '3578', '3579', '3580', '3582', '3585', '3597', '3598', '36', '3601', '3602', '3603', '3604', '3605', '3613', '3619', '3620', '3623', '3624', '3626', '3629', '3630', '3631', '3633', '3634', '3637', '3643', '3647', '3649', '3651', '3667', '3668', '3669', '3672', '3674', '3676', '3680', '3681', '3688', '3695', '3697', '3699', '37', '3713', '3721', '3730', '3734', '3738', '3740', '3742', '3743', '3750', '3751', '3755', '3757', '3760', '3763', '3764', '3765', '3768', '3771', '3772', '3774', '3777', '378', '3782', '3784', '3785', '3786', '3787', '3788', '38', '3801', '3806', '3807', '3808', '381', '3817', '3831', '3840', '3841', '3846', '3850', '3851', '3853', '3857', '3858', '3860', '3865', '3867', '3870', '3872', '3874', '3875', '388', '3884', '3886', '3890', '3893', '3894', '3895', '3899', '390', '3900', '3903', '3905', '3908', '3909', '3917', '392', '3922', '3937', '3944', '3946', '3951', '3952', '3954', '3958', '396', '3968', '3971', '3977', '3989', '399', '3994', '3996', '40', '4000', '4012', '4014', '4015', '4019', '4021', '4023', '4025', '4029', '404', '405', '406', '4065', '4067', '407', '4074', '4075', '4081', '4085', '4087', '4089', '409', '4092', '4094', '4097', '4098', '41', '410', '4100', '4105', '4106', '4111', '4120', '4123', '4132', '4141', '4142', '4143', '4144', '4149', '4155', '4157', '4160', '4169', '4175', '4176', '4177', '4178', '4184', '4185', '4189', '4192', '4193', '4194', '420', '4205', '4210', '4216', '4218', '4219', '422', '4220', '4223', '4228', '423', '4232', '4239', '4240', '4241', '4245', '4253', '4257', '4258', '4261', '4266', '4273', '4276', '4279', '4285', '4286', '4288', '4291', '4292', '4302', '4304', '4307', '4313', '4318', '4327', '4330', '4331', '4337', '4338', '4341', '4344', '4346', '4347', '435', '4351', '4353', '4357', '4358', '4361', '4363', '4364', '4370', '4372', '4374', '4376', '4378', '438', '4380', '4384', '4385', '4387', '4388', '4389', '439', '4397', '4398', '4399', '4404', '4410', '4412', '4419', '4420', '4433', '4437', '4438', '4439', '4440', '4446', '4451', '4452', '4454', '4457', '4459', '4460', '4468', '4472', '4475', '4476', '449', '4494', '4498', '4504', '4505', '4517', '4521', '4523', '455', '4551', '4554', '456', '4560', '4562', '4574', '4581', '4583', '4585', '461', '4610', '4613', '4620', '4621', '464', '4644', '4649', '4652', '4657', '466', '4687', '4691', '4692', '4695', '470', '4729', '473', '4732', '4745', '4747', '4751', '4755', '4756', '4759', '4765', '4773', '4775', '4776', '4777', '4779', '4780', '4782', '4788', '4791', '4792', '4794', '4795', '4796', '4797', '480', '4801', '4803', '4804', '4805', '4810', '4814', '4819', '4820', '4821', '4827', '483', '484', '4843', '4892', '490', '4900', '4901', '4911', '4914', '4920', '4922', '4930', '4934', '494', '4954', '4956', '4960', '4972', '4974', '4990', '4991', '4998', '4999', '50', '5012', '5013', '5015', '5018', '5021', '5029', '5034', '5042', '5043', '5046', '5048', '5050', '5051', '5053', '5054', '5055', '5058', '506', '5062', '5068', '5071', '5072', '5076', '5078', '508', '5083', '5084', '5085', '5086', '5089', '509', '5090', '5092', '5095', '5100', '5105', '5114', '512', '5126', '5128', '513', '514', '517', '5172', '5188', '521', '5210', '5219', '5222', '5228', '524', '5250', '5257', '5264', '5276', '528', '5283', '529', '5295', '5296', '5297', '53', '530', '5300', '5308', '531', '5312', '5313', '5314', '5316', '5317', '5321', '5326', '5327', '5328', '533', '5339', '534', '5342', '5345', '5346', '535', '5351', '5354', '5355', '5356', '5358', '5359', '536', '5366', '5367', '5377', '538', '5385', '539', '5390', '5402', '5407', '541', '5410', '5414', '5415', '5416', '5417', '5422', '5423', '5426', '5429', '5431', '5437', '5438', '5440', '5443', '5444', '5451', '5452', '5454', '5456', '546', '5461', '5464', '5470', '5473', '5474', '5475', '548', '5483', '5492', '5494', '5495', '5496', '5500', '5501', '5502', '5511', '552', '5522', '5525', '5528', '5530', '5532', '5535', '5537', '5538', '554', '5540', '5543', '5544', '5545', '5546', '5549', '555', '5555', '5556', '5563', '5565', '5566', '5568', '557', '5571', '5572', '5573', '5575', '5576', '5579', '5582', '5583', '5585', '5586', '5592', '5593', '5594', '5595', '56', '560', '5600', '5602', '5603', '5604', '5617', '5620', '5624', '5627', '5629', '5632', '5644', '5647', '5650', '5654', '5657', '5658', '5660', '5662', '5663', '5669', '5677', '5678', '5679', '5680', '5682', '5687', '5688', '5689', '5694', '5701', '5707', '5711', '5712', '5715', '5716', '5719', '5720', '5721', '5722', '5723', '5725', '5727', '5729', '5731', '5737', '5738', '5741', '5742', '5743', '5744', '5746', '5749', '5758', '5787', '5815', '5822', '5894', '5898', '5901', '5906', '5911', '5912', '5913', '5914', '5922', '5925', '5926', '5930', '5931', '5935', '5938', '5950', '5951', '5960', '5965', '5967', '5968', '5969', '5976', '5982', '5984', '5991', '5993', '5994', '6005', '6010', '6013', '6014', '6017', '6018', '6033', '6035', '6041', '6042', '6043', '6047', '6048', '6051', '6054', '6055', '6073', '6075', '6078', '6084', '6088', '6094', '6102', '6104', '6105', '6106', '6107', '6113', '6115', '6123', '6129', '6130', '6134', '6138', '6146', '6147', '6149', '6154', '6155', '6165', '6172', '6173', '6175', '6179', '6181', '6185', '6186', '6187', '6198', '6202', '6204', '6205', '6211', '6213', '6214', '6215', '6220', '6222', '6227', '6231', '6232', '6233', '6234', '6235', '6240', '6242', '6243', '6244', '6248', '6252', '6254', '6257', '6271', '6273', '6274', '6275', '6279', '6280', '6283', '6287', '6288', '6293', '6295', '6297', '6298', '6299', '6305', '6306', '6309', '6310', '6311', '6312', '6313', '6323', '6324', '6325', '6333', '6335', '6341', '6347', '6349', '6353', '6355', '6356', '6358', '6372', '6376', '64', '6419', '6420', '6434', '6437', '6438', '6440', '6447', '6454', '6455', '6456', '6457', '6458', '6459', '6460', '6461', '6465', '6466', '6471', '6474', '6475', '6476', '6478', '6481', '6485', '6491', '6495', '6496', '65', '6500', '6503', '6504', '6505', '6506', '6507', '6509', '6514', '6520', '6521', '6522', '6523', '6524', '6525', '6528', '6529', '6532', '6533', '6534', '6539', '6540', '6542', '6547', '6549', '6551', '6552', '6559', '6568', '6572', '6573', '6575', '6577', '6581', '6583', '6584', '6586', '6590', '6591', '6592', '6594', '6597', '66', '6601', '6607', '6609', '6611', '6612', '6620', '6622', '6629', '6635', '6636', '6641', '6642', '6647', '6653', '6661', '6668', '6669', '6670', '6672', '6673', '6674', '6679', '6680', '6681', '6683', '6684', '6685', '6686', '6687', '6696', '6699', '6701', '6705', '6708', '6709', '6711', '6713', '6721', '6723', '6726', '6729', '6730', '6732', '6733', '6734', '6737', '6738', '6739', '6740', '6741', '6742', '6743', '6747', '6748', '6752', '6759', '6761', '6762', '6763', '6765', '6770', '6771', '6773', '6774', '6776', '6777', '6778', '6779', '6783', '6784', '6785', '6787', '6790', '6794', '6797', '6799', '680', '6801', '6802', '6809', '6813', '6814', '6816', '6825', '6827', '6830', '6831', '6837', '6840', '6848', '6850', '6852', '6859', '686', '6860', '6861', '6862', '6863', '6864', '6865', '687', '6875', '6877', '6880', '6881', '689', '6894', '690', '6907', '691', '6917', '692', '693', '6931', '695', '696', '6969', '6972', '698', '6986', '699', '6999', '700', '7000', '7001', '7003', '7008', '701', '702', '7048', '7052', '7058', '7060', '7061', '7063', '7064', '7066', '707', '7072', '7073', '7074', '7075', '7076', '7077', '7078', '7079', '7083', '7085', '7092', '7096', '71', '7102', '7106', '7107', '7113', '7115', '7120', '7123', '7128', '7132', '7133', '7139', '7141', '7146', '7148', '715', '7155', '7157', '7158', '7167', '7174', '7184', '7185', '7189', '7190', '7191', '7192', '7193', '7208', '7209', '7212', '7216', '7217', '722', '7226', '7229', '7235', '7244', '7245', '7250', '7256', '7259', '7261', '7264', '7266', '7267', '7272', '7275', '7276', '7277', '7278', '7279', '728', '7286', '7297', '730', '7302', '731', '7317', '7320', '7322', '7329', '7330', '7331', '7332', '7338', '7339', '734', '7341', '7346', '7349', '7358', '7359', '7363', '7366', '7379', '7381', '7382', '7385', '7388', '739', '7398', '74', '7404', '7410', '7413', '7424', '7427', '743', '7432', '7438', '7439', '7441', '7442', '7443', '7444', '7449', '7455', '7459', '7462', '7465', '7466', '7468', '7472', '7474', '7476', '7481', '7483', '7487', '7488', '7491', '7495', '7498', '7505', '7508', '7514', '7516', '7517', '7518', '7519', '7520', '7541', '7544', '756', '7576', '758', '7609', '7620', '7621', '7622', '7631', '7634', '7636', '7647', '7657', '7673', '7676', '7681', '7682', '7686', '7687', '7690', '7694', '7696', '7702', '7707', '7709', '7722', '7723', '7725', '7726', '7728', '773', '7730', '7732', '7734', '7735', '7736', '7738', '7739', '774', '7742', '7743', '7746', '7752', '7755', '7757', '776', '7761', '7762', '7763', '7766', '7768', '7774', '7776', '778', '7782', '7790', '7797', '780', '7805', '781', '7821', '7826', '7828', '783', '7830', '7835', '7836', '7837', '7838', '7840', '7841', '7843', '7845', '7848', '7850', '7851', '7857', '7858', '7859', '7861', '7864', '7866', '7869', '7870', '7871', '7872', '7874', '7875', '7876', '7878', '7879', '7880', '7881', '7883', '7884', '7887', '7893', '7894', '7901', '7902', '7903', '7904', '7913', '7914', '7918', '7922', '7927', '7939', '7942', '7948', '7949', '7953', '7954', '7955', '7956', '7958', '7970', '7974', '7975', '7979', '7987', '7990', '7991', '7994', '8000', '8001', '8005', '8007', '8011', '8012', '8016', '8017', '8022', '8025', '8026', '8027', '8028', '8030', '8032', '8036', '8039', '8045', '8047', '8048', '8050', '8051', '8056', '8087', '8094', '8096', '811', '8120', '8125', '8155', '8162', '8163', '8169', '817', '8177', '818', '82', '8218', '8219', '8221', '8223', '8226', '8227', '8229', '8236', '8245', '8253', '8254', '8258', '8262', '8269', '8270', '8275', '8276', '828', '8286', '8289', '8291', '8292', '8294', '8296', '8297', '83', '8300', '8311', '8313', '8314', '8317', '8318', '8320', '8322', '8334', '8335', '8337', '8338', '8339', '8340', '8349', '8350', '8364', '8366', '837', '8378', '8388', '8391', '8394', '8399', '8407', '8409', '8424', '8425', '8426', '8435', '8447', '8448', '8449', '8454', '8457', '8458', '846', '8475', '8479', '848', '8480', '8481', '8487', '8488', '8489', '8490', '8491', '85', '8501', '8502', '8506', '8510', '8513', '852', '8520', '8521', '8526', '8528', '8530', '8533', '8534', '8540', '8541', '8545', '8547', '8555', '8556', '8559', '8561', '8562', '8565', '8569', '8577', '8579', '8580', '8581', '8582', '859', '8595', '86', '8600', '8609', '861', '8611', '8612', '8614', '8616', '8619', '8630', '8632', '8643', '8644', '865', '866', '8665', '8668', '868', '87', '8702', '8707', '8708', '8716', '8722', '8724', '8731', '8732', '8733', '8736', '8738', '8740', '8741', '8742', '8745', '8750', '8761', '8767', '8768', '8777', '8784', '8785', '8788', '879', '8791', '8793', '8794', '8797', '8799', '8806', '8808', '8810', '8814', '8820', '8826', '883', '8837', '8846', '8858', '8859', '886', '8860', '8861', '8863', '8865', '8866', '8868', '887', '8879', '8890', '8892', '8893', '8898', '8899', '89', '8902', '8905', '8908', '8912', '8915', '8925', '8928', '8930', '8935', '8940', '8946', '8956', '8959', '8964', '8965', '8968', '8969', '8970', '8971', '8972', '8974', '8976', '8979', '8981', '8985', '8986', '899', '8992', '8995', '8997', '8999', '9', '9003', '9029', '9030', '9034', '9036', '9040', '9044', '9048', '9049', '9053', '9056', '9066', '9072', '9073', '9074', '9091', '9097', '9101', '9103', '9111', '9112', '9113', '9115', '9116', '9117', '9127', '9139', '9142', '9158', '9159', '9160', '9161', '9175', '9177', '9181', '9182', '9187', '9189', '9192', '920', '9202', '9206', '921', '922', '9228', '9229', '9231', '9232', '9235', '9237', '9238', '9240', '9242', '9250', '9255', '9260', '9263', '9266', '9267', '9268', '9270', '9271', '9272', '9273', '9274', '9278', '9280', '9284', '9287', '93', '9301', '9306', '9311', '9312', '9313', '9316', '9328', '9330', '9333', '9334', '9335', '9337', '9339', '9346', '9352', '9354', '9364', '9366', '9369', '9371', '9372', '9373', '9378', '938', '9383', '9385', '9388', '9389', '9399', '9403', '9405', '9408', '9413', '9414', '9417', '9422', '9425', '9426', '9427', '9428', '9429', '9437', '9438', '9441', '9444', '9463', '9465', '9475', '9477', '9478', '9484', '9486', '9488', '949', '9491', '9502', '9503', '9505', '9511', '9519', '9525', '9530', '9531', '9543', '9547', '9549', '9551', '9556', '9558', '9560', '9562', '9563', '9566', '9572', '9576', '9577', '9578', '958', '9580', '9587', '9588', '9590', '9598', '9599', '9605', '9606', '9610', '9611', '9612', '9613', '9618', '9625', '9641', '9647', '9652', '9671', '9678', '968', '9682', '9706', '9707', '971', '9723', '9728', '9734', '9736', '9741', '9744', '976', '9760', '9761', '9771', '98', '9804', '9807', '981', '9810', '9815', '9818', '9825', '9827', '983', '9837', '9839', '984', '986', '9860', '9861', '9868', '9869', '9874', '9875', '9878', '988', '9884', '9894', '9896', '9901', '9902', '9904', '9909', '9914', '9927', '9930', '9934', '9936', '9937', '9939', '9940', '995', '9952', '9956', '9959', '9961', '9963', '9964', '9965', '9967', '9970', '9973', '998', '9981', '9985', '9992', '9993', '9994', '9995'],
    'fuel': ['10860', '11070', '12315', '14732', '15924', '16314', '1660', '17441', '17473', '18228', '18480', '1856', '18858', '18902', '19497', '20406', '2957', '3995', '5630', '5683', '6798', '7397', '8041'],
    'gas': ['10038', '11406', '11491', '11699', '11880', '12209', '14698', '14704', '14732', '14863', '1520', '15339', '15344', '15351', '16581', '1660', '17441', '17446', '17463', '17472', '17878', '18146', '18311', '18367', '18480', '1851', '1856', '1878', '19083', '19664', '20730', '2828', '3855', '3995', '4507', '5037', '5061', '5145', '6836', '6871', '6893', '7150', '7152', '7671', '7854', '7972', '829', '8882', '8964', '9039', '9614', '9634', '9912', '9915'],
    'gnp': ['10135', '10289', '10340', '1049', '10531', '1058', '1068', '10827', '1102', '11083', '11191', '11291', '11327', '11356', '11776', '11789', '11795', '11882', '11949', '12461', '12576', '1272', '12771', '12788', '12986', '13258', '13274', '14891', '15551', '16108', '1611', '16158', '16171', '16196', '16197', '16201', '16226', '16268', '17883', '17896', '179', '17906', '17970', '18009', '18095', '18490', '1858', '18911', '18985', '18987', '18990', '18991', '18993', '18994', '18995', '18996', '19000', '19004', '19005', '19017', '19061', '19075', '19478', '1959', '209', '21368', '21525', '2197', '227', '2278', '2618', '2709', '2843', '2965', '3019', '3024', '3031', '3065', '3246', '3351', '3352', '3411', '3800', '3838', '4080', '4165', '4297', '4548', '4573', '4625', '4944', '5206', '5271', '5447', '5453', '5458', '5472', '5783', '5803', '5804', '5805', '5807', '5849', '5879', '5980', '5985', '6037', '6197', '6384', '6406', '6416', '6417', '6427', '6494', '6593', '6621', '6758', '6942', '6943', '6959', '7032', '7222', '7581', '7583', '8111', '8135', '8153', '8164', '8172', '8173', '8347', '843', '8453', '8746', '9196', '9769'],
    'gold': ['10214', '10216', '10485', '10546', '10662', '1072', '10811', '1082', '10868', '10969', '10994', '11627', '11638', '11830', '11839', '11852', '12442', '12796', '12963', '12987', '13116', '13690', '13757', '13791', '14572', '14842', '15411', '15471', '15481', '15803', '15811', '16009', '1607', '16149', '16162', '16212', '16248', '16286', '16589', '16604', '17457', '17622', '17632', '17654', '17714', '18013', '18176', '1897', '19152', '1918', '1942', '19764', '19802', '19808', '20096', '20699', '2073', '21293', '21314', '21576', '2411', '2559', '2762', '2782', '2785', '2880', '2983', '309', '314', '3322', '3327', '3574', '3625', '3646', '3704', '3793', '3997', '4058', '4066', '4407', '448', '4648', '4708', '4816', '488', '5209', '5255', '5323', '5481', '5526', '5541', '5558', '5564', '5567', '5932', '5958', '6218', '6658', '6872', '6957', '697', '6985', '7023', '7088', '7166', '7629', '7645', '7775', '7823', '7985', '8068', '8331', '8757', '8854', '8857', '8877', '8948', '9104', '9190', '9453', '9489', '9799', '9853', '9866'],
    'grain': ['10120', '10124', '10139', '10172', '10175', '10268', '10305', '10319', '10339', '10406', '10444', '10487', '10489', '10494', '10519', '10569', '1057', '106', '10643', '1067', '1069', '10701', '10830', '10882', '1094', '10956', '11012', '11040', '11065', '11085', '11091', '11119', '11208', '11216', '11224', '11269', '1131', '11316', '11343', '11380', '11392', '11399', '11415', '11427', '11436', '11445', '11536', '11607', '11608', '11610', '11612', '11640', '11645', '11655', '11671', '11694', '11729', '11739', '11768', '11769', '11815', '11834', '11840', '1185', '11862', '11885', '11886', '11911', '11936', '11939', '11964', '12002', '12009', '12052', '12055', '12081', '1215', '12160', '12161', '12192', '12254', '12261', '12269', '12311', '12323', '12325', '12327', '12338', '12366', '12372', '124', '12417', '12420', '12428', '12436', '12458', '12500', '12543', '1257', '12583', '12587', '1268', '1273', '12762', '12768', '12802', '12830', '12849', '12872', '12887', '1295', '12971', '1299', '13067', '13070', '13094', '13097', '13099', '13120', '13123', '13129', '13162', '13173', '13179', '13272', '13294', '13313', '13458', '136', '1369', '13729', '1377', '13795', '13852', '13856', '1388', '1395', '1396', '1399', '1405', '1406', '14126', '14212', '14313', '14340', '14360', '14389', '14483', '14491', '14509', '14515', '14688', '14701', '14706', '14718', '14721', '14722', '14828', '14832', '14841', '14858', '15033', '15043', '15097', '15106', '15132', '15271', '15273', '15287', '15303', '15341', '15367', '15388', '15472', '15500', '15531', '15567', '1557', '15572', '15580', '15582', '15618', '15648', '15649', '15676', '15686', '1570', '15710', '15720', '15726', '15728', '1582', '15836', '15842', '15844', '15845', '15853', '15856', '15860', '15863', '15864', '15866', '15868', '15869', '15870', '15871', '15874', '15875', '15877', '15890', '15893', '15895', '1590', '15904', '15906', '15910', '15911', '15914', '15916', '15917', '15927', '15928', '15942', '15952', '15975', '15999', '16009', '16012', '16014', '16071', '16079', '16099', '16144', '16147', '1623', '16265', '16271', '1631', '16359', '1640', '1652', '16525', '16601', '16624', '1674', '16751', '16753', '16763', '16765', '16790', '1731', '17443', '17503', '17509', '17521', '17662', '17722', '17755', '17767', '17769', '1777', '1792', '17955', '17962', '180', '18024', '18035', '18263', '18275', '18297', '18306', '18345', '1843', '1845', '18482', '18609', '18614', '18642', '1882', '18899', '18908', '18954', '18973', '19', '1901', '19055', '19059', '1907', '1915', '19165', '19169', '19213', '19275', '193', '19322', '19388', '1952', '19549', '19668', '197', '1970', '19721', '19821', '19835', '19947', '19964', '20018', '2003', '20052', '2008', '20231', '20239', '20258', '20366', '2044', '20511', '20550', '20611', '20637', '20645', '20649', '20723', '20738', '2074', '20763', '2095', '21091', '21123', '21152', '21168', '21243', '21368', '21493', '21530', '21570', '2169', '2171', '2172', '2183', '2191', '2217', '2223', '2232', '2264', '228', '235', '2367', '2382', '241', '2425', '2436', '2456', '2508', '2524', '253', '2535', '254', '2595', '2599', '2617', '2727', '2741', '2747', '2749', '2777', '2848', '2857', '2864', '2913', '2922', '2947', '2975', '3082', '3084', '3132', '3138', '3191', '327', '3272', '3282', '3299', '3306', '3311', '3312', '3314', '3323', '3324', '3330', '3334', '3335', '3337', '3358', '3386', '3390', '3401', '3429', '3445', '3737', '3847', '3855', '3881', '3897', '3904', '3913', '3949', '395', '3979', '3981', '4047', '4057', '4077', '4133', '417', '4203', '425', '4278', '4280', '4289', '4296', '4314', '4356', '4382', '4473', '4478', '4490', '4524', '4549', '4599', '4637', '4671', '4678', '4696', '4719', '4778', '4825', '4905', '4939', '4988', '5', '5003', '501', '5017', '5033', '5056', '5109', '516', '5185', '5266', '5338', '5363', '5383', '5408', '5467', '5518', '5531', '5606', '5610', '5611', '5636', '5637', '5639', '5640', '5648', '57', '5800', '5826', '5847', '5933', '5972', '6', '6006', '6079', '6081', '6114', '6127', '6142', '6153', '6209', '6221', '6236', '6253', '6259', '6265', '6267', '6269', '6296', '6326', '6339', '6386', '6585', '6588', '6626', '6720', '6735', '6744', '6751', '6882', '6890', '6897', '6898', '6903', '6909', '694', '6946', '6960', '7062', '7103', '7154', '7160', '7205', '7215', '7326', '7336', '7387', '7389', '7390', '7395', '742', '7471', '7545', '7565', '7579', '7700', '7792', '7917', '7934', '7943', '7957', '8003', '8004', '8009', '8132', '8140', '8161', '8164', '8166', '8179', '8213', '8257', '8273', '8319', '8374', '8400', '8413', '8443', '8446', '8535', '855', '856', '8604', '8613', '8656', '8686', '874', '8759', '8800', '8895', '8941', '8943', '8983', '8993', '9018', '9021', '9058', '9093', '9094', '9095', '934', '9406', '9470', '9521', '9667', '97', '9705', '9772', '9773', '9782', '9793', '9865', '9907', '9958', '9989'],
    'groundnut': ['12254', '12275', '12372', '14951', '15574', '16147', '19101', '2359', '2420'],
    'groundnut-oil': ['15500', '313'],
    'heat': ['10567', '11491', '11880', '12935', '14558', '14698', '15038', '15520', '15607', '15829', '19223', '2696', '3181', '3310', '5061', '5706', '7397', '829', '834'],
    'hog': ['10857', '11914', '15532', '16255', '17823', '17827', '19555', '1970', '21367', '2226', '2572', '3057', '3278', '3318', '3595', '4206', '4833', '5333', '7101', '798', '9060', '9585'],
    'housing': ['1035', '1036', '11170', '11665', '18911', '19875', '20106', '20116', '29', '3105', '3708', '3720', '3723', '3898', '5883', '5886', '6000', '6067', '6197', '9615'],
    'income': ['10602', '10604', '16118', '18534', '18540', '18664', '18665', '18672', '18911', '2618', '7005', '7006', '7015', '7036', '7098', '7099'],
    'instal-debt': ['14566', '14612', '18296', '3016', '3341', '3353'],
    'interest': ['1008', '10091', '10156', '10246', '10277', '10397', '10464', '10503', '10545', '1078', '11083', '1112', '1120', '11200', '11238', '11244', '1125', '11263', '11314', '11365', '11422', '11425', '1154', '11660', '11716', '11746', '11776', '11817', '11820', '11831', '11848', '11858', '11861', '11900', '11903', '11907', '11913', '11934', '11940', '11941', '11960', '11989', '12010', '12016', '12019', '12027', '12040', '12073', '12084', '12091', '12101', '12102', '12112', '12122', '12125', '12131', '12147', '12183', '12197', '12229', '12235', '12246', '12251', '12259', '12260', '12274', '12278', '12302', '12316', '12333', '12336', '12337', '12396', '12398', '12406', '12410', '12418', '12447', '12453', '12455', '12456', '12469', '12470', '12471', '12483', '12494', '12495', '12501', '12509', '12514', '12554', '12571', '12592', '12731', '12732', '12741', '12745', '12774', '12789', '12805', '12810', '12812', '12819', '12840', '12846', '12880', '12882', '12891', '12908', '12912', '12919', '12923', '13008', '13024', '13105', '13144', '13230', '13231', '13262', '13317', '13321', '13543', '1421', '14214', '1427', '14270', '14293', '14642', '14849', '14861', '14890', '14919', '14964', '1503', '15049', '15092', '15096', '15212', '15310', '15364', '15378', '15384', '15410', '15436', '15444', '15522', '1553', '15539', '15550', '15552', '15560', '15578', '15579', '1560', '15603', '15610', '15615', '15617', '15805', '15816', '16053', '16072', '16074', '16075', '16083', '16096', '16120', '16150', '16161', '16180', '16214', '16274', '16298', '16304', '16383', '16407', '16565', '16681', '16852', '17051', '17052', '1724', '17445', '17448', '17470', '17577', '17593', '17620', '17758', '17881', '17883', '17899', '17939', '17943', '17966', '18011', '18051', '18086', '18106', '18142', '18185', '18362', '18465', '18488', '18520', '18538', '18549', '18564', '18595', '18670', '18672', '18724', '18750', '18763', '18904', '19020', '19022', '19121', '19167', '19191', '19201', '19229', '19237', '1931', '1943', '19483', '19487', '19511', '19512', '19529', '19545', '19547', '19557', '1967', '19689', '19693', '1971', '1976', '19775', '1982', '19875', '1995', '20021', '20032', '20038', '20048', '20145', '20153', '20159', '2022', '20262', '20263', '20265', '20275', '20343', '20409', '20500', '20532', '20631', '2064', '20678', '20759', '20764', '20769', '21124', '21285', '21331', '21342', '21468', '21508', '21510', '21511', '2155', '221', '2228', '225', '233', '2358', '2576', '2601', '2626', '2678', '2683', '2697', '2709', '2725', '2925', '2952', '296', '2966', '2979', '2996', '3002', '3006', '3007', '3020', '3043', '3044', '3060', '3068', '307', '3087', '3088', '3092', '3103', '3111', '3121', '3122', '3128', '3131', '3133', '3134', '3137', '3164', '3192', '3219', '322', '3253', '3283', '3292', '336', '3360', '3365', '3371', '341', '3448', '3449', '3493', '3504', '3524', '3528', '3534', '3539', '3542', '3702', '3762', '3825', '3982', '4005', '4090', '411', '4131', '4188', '4275', '4298', '4409', '4449', '4455', '4458', '4483', '4484', '4499', '4602', '4656', '4682', '4720', '475', '4763', '4783', '4822', '4881', '4925', '4944', '5127', '519', '522', '5232', '5235', '5237', '5241', '5277', '5290', '5336', '5344', '5404', '5439', '5779', '5814', '5836', '5861', '6044', '6046', '6091', '6156', '6176', '6178', '6197', '6350', '6352', '6359', '6362', '6364', '6365', '6366', '6369', '6398', '6399', '6400', '6406', '6426', '6452', '6453', '6483', '6531', '6543', '6764', '6922', '7031', '7159', '7207', '7310', '7313', '7321', '7460', '7499', '7538', '7559', '7562', '7564', '7571', '7587', '7599', '7614', '7638', '7652', '7716', '7764', '7769', '8103', '8135', '8137', '8144', '8145', '8147', '8153', '8168', '8182', '8184', '8194', '8247', '8299', '8343', '8344', '8401', '8673', '8677', '8723', '8802', '8884', '8951', '8991', '900', '901', '9012', '913', '9137', '9138', '9164', '9216', '926', '9261', '9361', '942', '943', '953', '9656', '9675', '9701', '9730', '9735', '9784', '9841', '9848', '9855', '9943', '9972', '9988', '999'],
    'ipi': ['10230', '10283', '10285', '10314', '10520', '1079', '11379', '1158', '12403', '12407', '12466', '12771', '13238', '13559', '14854', '16142', '16843', '16853', '16897', '17897', '18536', '19077', '21207', '21368', '21423', '21575', '2618', '2990', '2995', '4005', '450', '4770', '4771', '4944', '4993', '5052', '5178', '5196', '5240', '5845', '5848', '6158', '6375', '7606', '7623', '7744', '8063', '8180', '8190', '8363', '8585', '8661', '8671'],
    'iron-steel': ['10043', '10268', '10275', '10315', '10406', '11030', '11039', '11104', '11107', '11121', '11124', '11179', '1156', '11706', '11813', '11918', '12046', '12533', '15262', '15313', '15623', '15639', '16122', '16706', '17674', '17829', '18254', '18752', '18768', '18951', '1934', '20318', '20341', '2982', '3469', '4016', '4049', '4215', '5148', '5153', '525', '6535', '6968', '7161', '7323', '7557', '7640', '8113', '8342', '9014', '9146', '9205', '9554', '9592'],
    'jet': ['20031', '2957', '6828', '7397', '9573'],
    'jobs': ['10135', '1040', '11159', '11160', '11276', '12507', '12534', '12555', '12743', '12752', '12818', '14771', '14867', '14974', '15045', '15365', '15485', '16100', '16158', '17633', '18533', '18539', '18616', '18672', '18987', '18990', '18993', '18996', '19000', '19985', '19986', '2000', '20248', '21525', '2197', '2618', '2746', '3024', '3472', '3520', '3717', '4036', '4091', '4306', '4698', '4700', '5215', '5392', '6158', '6603', '6951', '7004', '7010', '7013', '7070', '8173', '867', '8681', '8746', '889', '895', '955', '9751', '9805', '9812', '9834', '9897'],
    'l-cattle': ['15290', '20620', '2389', '3278', '443', '4564', '5896', '6492'],
    'lead': ['10485', '11935', '11958', '12484', '14572', '14863', '15556', '15563', '15767', '16289', '17480', '17486', '17783', '17805', '18337', '18571', '19431', '19477', '19764', '2186', '3211', '3454', '5153', '6451', '7775', '7809', '7842', '8854', '8877'],
    'lei': ['1023', '11259', '11261', '11356', '11546', '16281', '16310', '16767', '3531', '3621', '3640', '8178', '870', '972', '974'],
    'lin-oil': ['15500', '6'],
    'livestock': ['10124', '10616', '10857', '1110', '11914', '11957', '11961', '121', '12257', '12332', '12372', '12376', '12737', '13149', '13380', '13382', '1372', '1393', '1398', '14434', '14483', '14858', '15237', '15254', '15290', '15504', '15532', '1557', '15642', '15798', '15949', '16009', '16236', '16255', '16316', '16379', '16519', '16564', '1772', '18482', '18676', '19269', '19555', '1970', '20462', '20511', '20620', '21367', '2168', '2226', '2389', '2448', '2495', '2572', '2594', '2851', '2881', '2978', '3057', '3084', '3278', '3318', '3339', '3340', '3595', '3901', '4022', '4101', '4206', '443', '4503', '4514', '4564', '4577', '4631', '4636', '4833', '5009', '5031', '5333', '5400', '5445', '5487', '5812', '5896', '7101', '7921', '7924', '7962', '798', '8008', '8574', '8705', '8744', '8800', '9060', '9290', '9585', '9754'],
    'lumber': ['11519', '12943', '12948', '14840', '17670', '17686', '18281', '19371', '20790', '2793', '3942', '5115', '5132', '7318', '7566', '8422'],
    'meal-feed': ['10175', '11840', '1215', '12361', '12375', '12753', '12787', '13539', '15500', '15853', '15871', '15875', '15890', '15903', '15906', '15911', '15922', '15953', '15975', '16146', '16265', '17477', '18177', '18482', '19388', '1970', '200', '20208', '20723', '2456', '2557', '267', '2913', '2974', '303', '3036', '3084', '310', '3135', '327', '5635', '5887', '6623', '69', '7565', '7951', '8140', '9201', '9756'],
    'money-fx': ['10048', '10075', '1008', '10081', '10091', '10136', '10191', '10263', '10308', '10321', '10323', '10337', '10340', '10344', '10354', '10356', '10357', '10358', '10359', '10364', '10368', '10370', '10382', '10416', '10418', '10475', '10503', '10546', '10606', '10617', '10618', '10622', '10633', '10636', '10638', '10650', '10651', '10652', '10653', '10654', '10657', '10658', '10659', '10660', '10661', '10679', '10681', '10684', '10689', '10696', '10704', '10710', '10718', '10731', '10734', '10736', '10749', '10751', '10755', '10756', '10762', '10766', '10769', '10770', '10771', '10773', '10774', '10780', '10782', '10796', '10804', '10807', '10808', '10809', '10859', '10864', '1088', '10913', '11000', '11059', '11103', '1112', '11120', '11135', '11193', '11200', '11203', '11209', '11212', '11228', '11234', '11238', '11244', '11256', '11263', '11281', '11284', '11297', '11322', '11365', '11417', '11430', '11447', '11450', '1154', '11551', '11734', '11753', '11764', '11772', '11774', '11776', '11783', '11794', '11798', '11820', '11832', '11844', '11864', '11938', '11944', '11946', '11972', '12121', '12136', '12145', '12177', '12183', '12396', '12406', '12410', '12413', '12416', '12418', '12422', '12437', '12447', '12453', '12456', '12470', '12471', '12494', '12501', '12522', '12523', '12554', '12588', '12593', '12741', '12745', '12780', '12785', '12789', '12791', '12795', '12801', '12806', '12812', '12815', '12819', '12840', '12842', '12877', '12923', '13053', '13243', '13244', '13245', '13261', '13270', '13315', '13398', '13404', '13512', '13524', '13530', '13531', '13544', '13552', '13660', '13739', '13831', '13869', '13903', '13919', '13949', '13950', '13958', '13988', '13994', '14034', '14055', '14150', '14178', '14207', '1421', '1457', '14655', '14686', '14757', '14767', '14770', '14777', '14779', '14785', '14849', '14861', '14890', '14913', '14919', '14931', '14964', '14987', '1503', '15048', '1514', '15212', '15234', '15253', '1533', '15364', '15375', '15378', '15431', '15436', '15442', '15444', '15448', '15449', '15450', '15452', '15453', '15460', '15510', '15522', '15523', '15527', '1553', '15539', '1560', '15615', '15621', '15625', '15636', '15656', '15677', '15689', '15694', '16004', '16006', '16009', '16053', '16066', '16067', '16068', '16069', '16072', '16106', '16111', '16133', '16143', '16161', '16177', '16180', '16188', '16189', '16190', '16213', '16251', '16285', '16331', '16383', '16565', '16743', '16744', '16754', '16770', '16903', '17041', '17044', '17047', '1724', '17523', '17577', '17620', '17742', '17748', '17871', '17877', '17915', '17930', '17952', '17980', '18010', '18012', '18050', '18051', '18086', '1811', '18136', '18142', '18185', '18360', '18363', '18370', '18625', '18668', '18674', '18724', '18750', '18763', '19021', '19023', '19028', '19040', '19061', '19201', '19229', '19237', '1926', '1931', '19418', '1943', '19489', '1949', '19537', '19541', '1967', '19689', '19693', '1976', '1982', '1983', '19869', '19989', '19991', '19993', '20001', '20009', '20012', '20013', '20068', '20071', '20080', '20081', '20082', '20083', '20087', '20089', '2022', '2025', '20259', '20263', '20343', '20347', '20348', '20377', '20496', '20500', '20631', '20678', '20764', '20795', '20858', '20862', '20868', '20893', '20907', '20922', '20925', '20941', '20948', '20958', '21023', '21122', '21128', '21138', '21151', '21157', '21163', '21172', '21202', '21277', '21303', '21323', '21339', '21343', '21468', '21477', '21496', '21508', '21510', '21511', '21512', '21521', '21535', '21539', '21542', '21556', '21561', '21573', '2178', '2181', '2190', '221', '2286', '2327', '2352', '2354', '2406', '2452', '247', '256', '2565', '2576', '2601', '2623', '2626', '2633', '2678', '2683', '2697', '2765', '296', '2966', '2993', '2996', '3001', '3002', '3020', '3031', '3044', '3060', '3065', '3067', '3068', '307', '3076', '3121', '3137', '3159', '3164', '3222', '3267', '3292', '3338', '336', '341', '3411', '3419', '3421', '3433', '3435', '3442', '3461', '3482', '3483', '3493', '3514', '3526', '3532', '3533', '3535', '3554', '3584', '3593', '3612', '3666', '3694', '3729', '3735', '4038', '4088', '4090', '4093', '411', '4131', '4139', '4170', '4188', '4275', '4293', '4298', '4616', '4633', '4666', '4675', '4680', '4682', '4703', '4709', '4712', '4720', '475', '4768', '4783', '4809', '4822', '4829', '4882', '5070', '5176', '5181', '5183', '519', '5190', '5193', '5201', '5204', '5206', '522', '5223', '5232', '5241', '5269', '5271', '5277', '5279', '5280', '5290', '5336', '5338', '5344', '5360', '5394', '5404', '5412', '5434', '5439', '5778', '5779', '5811', '5814', '5819', '5841', '5858', '5861', '5905', '5909', '5945', '5998', '6044', '6046', '6083', '6108', '6137', '6338', '6357', '6359', '6394', '6399', '6400', '6426', '6445', '6452', '6531', '6543', '6576', '6945', '6981', '7020', '7027', '7036', '7043', '7080', '7100', '7159', '7204', '7207', '7312', '7313', '7375', '7394', '7429', '7460', '7538', '7559', '7562', '7568', '7571', '7587', '7599', '7614', '7628', '7652', '7669', '7710', '7716', '7764', '7769', '7804', '7950', '8064', '8074', '8097', '8098', '8145', '8147', '8153', '8168', '8182', '8189', '8194', '8199', '8234', '8247', '8299', '8302', '8309', '8563', '8578', '8584', '8586', '8587', '8588', '8590', '8591', '8597', '8605', '8607', '8621', '8629', '8641', '8657', '8658', '8664', '8670', '8674', '8678', '8710', '8714', '872', '8723', '8728', '8735', '8760', '8796', '8845', '8884', '8922', '8984', '9007', '9022', '903', '9054', '9055', '9061', '9079', '9118', '9120', '9128', '9129', '913', '9131', '9132', '9134', '9138', '9149', '915', '9162', '9164', '9166', '9170', '9213', '9214', '9216', '9217', '9218', '9220', '9222', '9224', '926', '9261', '9282', '9295', '9299', '9361', '9377', '942', '943', '9498', '953', '9534', '9535', '9656', '9681', '9686', '9689', '9698', '9699', '9701', '9720', '9727', '9730', '9745', '9753', '9764', '9781', '9783', '9784', '9792', '9797', '9848', '9851', '9852', '9855', '9857', '9862', '9864', '9871', '9880', '9923', '9946', '9955', '9957', '9975', '999'],
    'money-supply': ['100', '10079', '101', '10153', '10154', '10155', '10176', '10177', '10178', '10183', '102', '10246', '103', '10592', '10594', '10708', '10737', '10746', '10754', '10757', '109', '111', '11765', '12429', '12576', '12748', '12758', '12841', '13010', '13019', '1308', '13143', '13157', '13158', '13159', '132', '133', '13468', '13471', '13949', '13994', '14573', '14770', '15242', '15246', '15552', '15950', '15951', '15976', '15977', '15978', '15987', '15989', '15996', '16063', '16083', '16250', '16268', '16282', '16650', '16755', '16779', '16908', '17036', '17042', '17051', '18487', '18537', '18946', '1898', '19028', '1912', '19284', '19294', '1933', '19416', '198', '20016', '20025', '20062', '20063', '203', '216', '220', '2403', '2441', '2442', '2465', '2466', '2468', '2554', '2686', '2743', '2744', '2858', '2870', '2872', '2994', '30', '3049', '3073', '3246', '330', '3348', '3524', '3661', '3930', '4487', '4515', '4516', '4519', '4532', '4533', '4534', '4544', '4602', '4686', '4929', '4938', '5059', '5060', '5207', '5251', '5253', '5782', '5797', '5798', '58', '5836', '6156', '6619', '6970', '7007', '7009', '7012', '7014', '7029', '7036', '7313', '7419', '7420', '7421', '7425', '7426', '7428', '7437', '7499', '7554', '7555', '7587', '76', '7674', '77', '7767', '78', '7807', '7818', '7971', '7980', '80', '81', '830', '839', '8636', '8723', '9107', '9178', '92', '94', '940', '95', '9700', '9752', '99'],
    'naphtha': ['17880', '18480', '19497', '19903', '6535', '7397'],
    'nat-gas': ['10011', '10190', '10228', '10521', '10567', '11093', '11231', '11350', '11466', '11632', '12312', '12313', '12728', '12851', '13102', '13130', '13702', '14829', '15322', '15416', '16007', '16166', '1619', '16238', '16438', '16499', '16593', '1661', '1709', '17492', '17516', '176', '17658', '17669', '17913', '18253', '18325', '18651', '18739', '18782', '18857', '18896', '18917', '18932', '19110', '19271', '19463', '19471', '2007', '20093', '20666', '21417', '2394', '2480', '2511', '2515', '2923', '3015', '3145', '3169', '3174', '3210', '3758', '3798', '4016', '4162', '4232', '4467', '4569', '4590', '4951', '4962', '5037', '5061', '544', '5793', '6425', '6606', '6650', '6656', '6660', '6712', '6722', '6742', '6760', '6876', '6905', '708', '7174', '7408', '7423', '8041', '8100', '8108', '8210', '8630', '8812', '8815', '8820', '9077', '9100', '9436', '9583', '9639', '9650'],
    'nickel': ['10678', '12484', '14297', '14928', '2782', '2880', '4769', '5693', '7126'],
    'nkr': ['11972', '19537', '19541'],
    'nzdlr': ['10931', '18625', '21561', '6945'],
    'oat': ['13852', '1405', '15871', '15875', '15952', '16071', '19275', '19668', '197', '417', '5', '855', '8759', '97'],
    'oilseed': ['10172', '10175', '10300', '10590', '10705', '10712', '10720', '10758', '10882', '11012', '11052', '11233', '1151', '11541', '11607', '11612', '11739', '11778', '11836', '11840', '11885', '11911', '11936', '11939', '11964', '11984', '12002', '12076', '12160', '12192', '12254', '12275', '12348', '12372', '12425', '12439', '12757', '12982', '13173', '13179', '13313', '13915', '1394', '1405', '14483', '14512', '14603', '14706', '14858', '14892', '15043', '15106', '15129', '15500', '15536', '15574', '15686', '15721', '1582', '15845', '15853', '15865', '15871', '15875', '15890', '15906', '15917', '15921', '15953', '15975', '15999', '16147', '16265', '16433', '16525', '17755', '1792', '18177', '18306', '18345', '18482', '18544', '1882', '19057', '19277', '19549', '19630', '1966', '19668', '19721', '19821', '200', '20208', '20461', '20550', '20645', '2087', '20911', '21080', '21091', '21497', '2172', '229', '2359', '2382', '2456', '2521', '297', '303', '3082', '313', '3206', '327', '3282', '3299', '3453', '3458', '3467', '3540', '3888', '3979', '3981', '4040', '4076', '4083', '4133', '4392', '4490', '4512', '4549', '4599', '486', '5391', '5531', '5606', '5610', '5702', '5761', '5833', '6', '6116', '6265', '6269', '6890', '69', '6906', '694', '7037', '7356', '7515', '7625', '7700', '7951', '8003', '8112', '8132', '8140', '8213', '8273', '8319', '833', '8443', '8522', '855', '9058', '9197', '9521', '9617', '97', '9756', '9919'],
    'orange': ['11071', '12737', '1350', '1361', '15675', '15846', '15847', '15854', '15855', '15929', '15960', '16627', '16630', '16784', '17443', '1761', '2390', '3098', '3109', '3149', '3315', '3892', '4133', '4541', '7357', '7920', '9327'],
    'palladium': ['15420', '2782', '2880'],
    'palm-oil': ['11778', '12585', '12746', '13080', '13179', '13738', '14833', '14840', '14892', '16193', '16362', '17477', '18781', '19088', '19367', '20911', '235', '274', '313', '320', '332', '3499', '3540', '4138', '4305', '4650', '4681', '5152', '5218', '5292', '5315', '6342', '6344', '6354', '6535', '6954', '8149', '8159', '885', '944'],
    'palmkernel': ['11778', '20911', '235'],
    'pet-chem': ['10581', '10886', '11625', '12404', '12439', '12770', '14689', '1493', '15063', '17563', '17580', '17585', '17743', '18191', '18217', '18807', '18852', '19397', '19903', '20710', '2371', '3120', '4054', '5150', '5421', '6561', '7452', '8088', '8971', '9032', '9077', '9204'],
    'platinum': ['15420', '16162', '16248', '18013', '18489', '18888', '19295', '2782', '2880', '309', '448', '9345'],
    'potato': ['15620', '15858', '15859', '1970', '3317', '8713'],
    'propane': ['10567', '13130', '17927', '19397', '21486', '829'],
    'rand': ['21535', '7043', '9336'],
    'rape-oil': ['10330', '10758', '15500', '16080', '18781', '320', '3950', '8159'],
    'rapeseed': ['10758', '11052', '1151', '11885', '12076', '12254', '12982', '13179', '13313', '14603', '15129', '15721', '16433', '18345', '18482', '19277', '19549', '19668', '20208', '2172', '313', '3206', '327', '4040', '4083', '4512', '8213'],
    'reserves': ['10262', '10531', '10634', '1174', '12416', '12433', '12435', '12480', '12801', '12817', '12829', '12875', '13255', '13690', '13791', '14207', '15441', '15768', '16250', '16775', '1773', '17932', '17933', '17966', '18109', '18321', '18524', '18527', '18672', '18749', '18798', '1885', '19040', '1913', '19198', '1940', '20127', '20264', '2098', '2110', '2686', '283', '284', '323', '3593', '3660', '4062', '4093', '4113', '416', '4548', '48', '5181', '5193', '5216', '5279', '5767', '5841', '6983', '7049', '7188', '7221', '7937', '8158', '8164', '8597', '8829', '896', '9133', '937', '9787', '9822', '9905'],
    'retail': ['15742', '16118', '2269', '2531', '2618', '4005', '4150', '4152', '4173', '4255', '4290', '444', '5205', '5254', '5256', '6955', '7117', '7129', '7447', '8173', '8451', '8701', '8706', '8746', '9974'],
    'rice': ['10519', '10830', '11216', '11536', '11640', '11655', '13123', '13129', '13294', '14515', '14832', '14858', '15206', '15367', '15567', '15871', '15875', '1590', '15906', '15928', '16079', '16099', '16147', '16601', '16784', '17962', '18609', '18642', '1882', '19059', '1915', '19165', '19388', '19835', '19947', '20052', '21570', '228', '235', '3358', '3445', '4133', '4314', '4549', '4678', '4719', '5266', '5610', '6114', '6153', '6269', '7160', '7545', '7917', '8161', '8413', '8759', '9018', '9705'],
    'rubber': ['10615', '10729', '10886', '11224', '11775', '12959', '1444', '14783', '14832', '14840', '1535', '15409', '15424', '16776', '17455', '17894', '20079', '20230', '20267', '21493', '21557', '235', '2977', '3051', '3155', '3438', '3748', '4405', '4824', '4941', '5138', '5139', '5195', '5792', '5957', '6677', '685', '6954', '7304', '7860', '7873', '8060', '8149', '8206', '8603', '8638', '884', '9153', '9203'],
    'rye': ['19668', '97'],
    'ship': ['10043', '10302', '10388', '10391', '10394', '10395', '10452', '1052', '10539', '10559', '10582', '10588', '106', '10620', '10621', '10627', '10641', '10717', '10748', '10797', '10803', '11251', '11271', '11385', '11587', '11694', '11813', '11840', '11885', '11911', '12081', '12161', '12162', '12164', '12174', '12179', '12249', '12270', '12327', '12366', '12371', '1257', '12761', '12768', '12770', '12823', '12849', '12924', '13120', '13162', '13179', '13247', '13320', '13542', '13688', '13963', '14211', '14360', '14491', '14537', '14682', '14688', '14701', '14709', '14818', '14839', '14957', '14959', '15484', '15531', '1556', '15696', '15710', '15726', '15727', '15728', '15942', '16014', '16040', '16076', '16366', '1723', '17436', '17462', '17492', '17750', '17888', '17925', '17955', '17979', '18128', '18160', '18213', '18231', '18271', '18328', '18329', '18340', '18357', '18470', '18521', '18571', '18851', '18865', '1901', '1902', '1904', '19055', '19073', '19092', '19097', '19122', '19199', '19238', '19403', '19491', '19684', '19756', '1980', '19882', '19996', '20030', '20034', '20092', '20101', '20103', '2012', '20261', '20333', '20444', '20464', '20624', '20719', '20756', '20774', '20828', '20881', '20882', '20890', '20902', '20981', '20991', '21002', '21006', '21013', '21149', '21216', '21371', '21443', '21459', '21475', '21482', '21485', '21486', '21492', '21501', '21506', '21517', '21523', '21526', '21574', '2187', '2310', '2326', '2542', '2635', '2681', '281', '2819', '2828', '2853', '2944', '2955', '2958', '2959', '2968', '2971', '3017', '3028', '3046', '3047', '3110', '3217', '3256', '3259', '3285', '3329', '3347', '3349', '3386', '3440', '3473', '3572', '368', '3796', '3830', '3869', '3955', '3973', '4049', '4051', '4061', '4171', '4203', '4209', '4328', '4345', '44', '4617', '4640', '4711', '4735', '4739', '4740', '4778', '4781', '4832', '4845', '49', '4953', '5154', '5156', '518', '5191', '5292', '5330', '5499', '5542', '5553', '5630', '5734', '5818', '5852', '5866', '5870', '59', '6404', '6436', '6541', '6588', '6652', '6714', '677', '6941', '7103', '7119', '7196', '7500', '7534', '7642', '7792', '7959', '7961', '8009', '8109', '8159', '8188', '8197', '8200', '8214', '8246', '8319', '8421', '8440', '8516', '857', '8675', '8688', '8696', '8744', '8749', '8795', '8856', '8902', '8950', '9314', '9348', '9356', '9362', '9473', '9479', '9604', '969', '9718', '9801', '9906'],
    'silver': ['10485', '11839', '11852', '12243', '12484', '12796', '13119', '14572', '16009', '16248', '17654', '17714', '18013', '18489', '1942', '19764', '21314', '3574', '3646', '3862', '3868', '488', '6515', '6557', '7645', '7775', '8854', '8877', '9195'],
    'sorghum': ['10175', '11091', '1131', '11609', '12052', '12160', '12417', '1369', '13852', '13856', '15043', '15500', '15567', '1582', '15871', '15875', '15917', '15952', '16071', '16753', '1843', '18482', '3981', '5', '5467', '5610', '5636', '6', '6890', '8140', '8446', '8759', '9521', '97'],
    'soy-meal': ['1215', '12361', '12425', '12753', '12787', '15853', '15871', '15875', '15890', '15903', '15906', '15911', '15922', '15953', '18177', '18482', '1970', '200', '20208', '20723', '235', '2456', '303', '3084', '327', '69'],
    'soy-oil': ['10175', '12425', '12786', '13173', '13179', '13915', '14892', '15500', '15751', '15853', '15871', '15875', '15890', '15906', '15923', '16080', '18781', '200', '2456', '313', '5142', '5791', '6', '6994', '833'],
    'soybean': ['10172', '10175', '10705', '10882', '11012', '11607', '11612', '11739', '11936', '11939', '11964', '11984', '12002', '12160', '12192', '12254', '12348', '12425', '12757', '13915', '1394', '1405', '14706', '14858', '14892', '15043', '15106', '15536', '15686', '1582', '15845', '15853', '15865', '15871', '15875', '15890', '15906', '15917', '15921', '15953', '15999', '16525', '16752', '17722', '1792', '18177', '18544', '1882', '19057', '19630', '19721', '19821', '200', '20208', '20335', '20461', '20645', '2087', '20911', '21080', '21091', '229', '235', '2382', '2456', '2521', '297', '303', '313', '3282', '3453', '3458', '3467', '3540', '3979', '4076', '4083', '4133', '4392', '4490', '4549', '4599', '5391', '5531', '5606', '5610', '5702', '6', '6062', '6116', '6265', '6269', '6890', '69', '6906', '694', '7037', '7356', '7515', '7625', '7700', '8003', '8140', '833', '8522', '855', '9058', '9521', '9617', '97', '9919'],
    'strategic-metal': ['10151', '11999', '12007', '13052', '13251', '14719', '15420', '15838', '15872', '17480', '17486', '17783', '17805', '18348', '18466', '18872', '18944', '2186', '2936', '2942', '3010', '309', '346', '3460', '3497', '5693', '7775'],
    'sugar': ['10014', '10107', '10124', '10299', '10306', '10367', '10373', '105', '10687', '10803', '11043', '11166', '11173', '11213', '11224', '11230', '11231', '11330', '11341', '11387', '11462', '11611', '11826', '11893', '12003', '12117', '12135', '1216', '12174', '12372', '12411', '12415', '12423', '12462', '12760', '12772', '12778', '12790', '14146', '1436', '1440', '14434', '14440', '14682', '14832', '14885', '14886', '14922', '14923', '14986', '14988', '15013', '15017', '1519', '1537', '15394', '15415', '15446', '15462', '15696', '15727', '15804', '1598', '16093', '16185', '16327', '16749', '17443', '17509', '1785', '17923', '18128', '1815', '18531', '18557', '18609', '18642', '19262', '19373', '1946', '19668', '1975', '20462', '21321', '21493', '21536', '21565', '2195', '2206', '2239', '2246', '235', '2359', '2389', '2521', '259', '2760', '2777', '293', '2964', '2975', '2976', '3191', '3228', '3310', '343', '3492', '3511', '3577', '3589', '3611', '3616', '3639', '3690', '3701', '3792', '3928', '4077', '4096', '4099', '4117', '46', '4630', '4898', '4965', '5175', '5214', '5835', '6484', '6486', '6657', '684', '6935', '6939', '6998', '7308', '7501', '7510', '7512', '7545', '7569', '8004', '8176', '8306', '8654', '8683', '8684', '8800', '9069', '9124', '9163', '9207', '9304', '9305', '9349', '9362', '9374', '9393', '9398', '9470', '9746', '9941'],
    'sun-meal': ['12361', '15500'],
    'sun-oil': ['10758', '15500', '18781', '3950', '6', '6412', '9193'],
    'sunseed': ['10758', '11964', '12160', '12254', '15043', '1582', '15917', '19630', '20208', '20550', '313', '3981', '4083', '6', '6890', '9521'],
    'tea': ['10268', '10406', '12754', '12907', '16225', '17494', '19672', '19982', '235', '275', '7545', '9153', '9327'],
    'tin': ['10332', '11224', '11801', '13185', '14832', '14844', '14877', '15112', '15219', '15624', '15817', '17731', '18924', '19065', '1929', '19367', '20458', '311', '4122', '688', '6934', '7533', '7592', '7877', '8055', '8415', '8416', '8427', '8933', '908'],
    'trade': ['10005', '10209', '1022', '10255', '10264', '10265', '10268', '10275', '10297', '10341', '10347', '10352', '10355', '10362', '10372', '10375', '10376', '10406', '10455', '10520', '10530', '10608', '10623', '10625', '10642', '10643', '10665', '10695', '10767', '10771', '10779', '10780', '10781', '10807', '10825', '10905', '11076', '11174', '11175', '11198', '11209', '11222', '11225', '11229', '11254', '11260', '11281', '11287', '11315', '11352', '11357', '11384', '11390', '11397', '11420', '11446', '11460', '11461', '11487', '11496', '11498', '11545', '11554', '11555', '11558', '11574', '11580', '11634', '11658', '11771', '1197', '1201', '1210', '1226', '12401', '12457', '12472', '12473', '12558', '12563', '12564', '12755', '12848', '13039', '13045', '13046', '1347', '13482', '1356', '13649', '13836', '13842', '13849', '13869', '13889', '13903', '13908', '13946', '14012', '14109', '14199', '14220', '14739', '14770', '1478', '14826', '14832', '14858', '14862', '14881', '14904', '14912', '1499', '15154', '15171', '15223', '15262', '15313', '15352', '15372', '15375', '15386', '15428', '15430', '15447', '15452', '15552', '15646', '15666', '15725', '15737', '15840', '1588', '16052', '16088', '16094', '16095', '1611', '16118', '16125', '16126', '16139', '16190', '16256', '16257', '16357', '16442', '16505', '1656', '16745', '1675', '16763', '16766', '16774', '16775', '16777', '16784', '16787', '16788', '16790', '16794', '1680', '16856', '16871', '16926', '16929', '16932', '17449', '17452', '17511', '17670', '1772', '17871', '17924', '17926', '17948', '18061', '1809', '18104', '1811', '18148', '18257', '18302', '18347', '1839', '18472', '18496', '18519', '18554', '1863', '1871', '18748', '18753', '18798', '18835', '18867', '1890', '18991', '18992', '19033', '19044', '19062', '19163', '19273', '19293', '1932', '19377', '1940', '19433', '19444', '1951', '19539', '19546', '19555', '19569', '1961', '1963', '1964', '19658', '19672', '19918', '19947', '20248', '20335', '20441', '20461', '20462', '20469', '20511', '20649', '20682', '20757', '20787', '20829', '20865', '20989', '21031', '21187', '21391', '21392', '21562', '2278', '2336', '2352', '2359', '2417', '2499', '2511', '2521', '2535', '2618', '2620', '2622', '2648', '271', '2862', '2985', '3019', '3065', '3078', '312', '3199', '3267', '3338', '338', '342', '3441', '3446', '3512', '3532', '354', '3553', '3593', '3610', '3615', '3690', '3770', '3902', '3931', '4026', '4027', '4031', '4048', '4078', '4084', '4114', '4115', '4156', '4158', '4172', '4238', '4259', '4480', '4509', '4513', '4552', '4595', '4629', '4632', '4635', '4638', '4654', '47', '4717', '4835', '4903', '4987', '5057', '5146', '5160', '5169', '5189', '5220', '5236', '5274', '5278', '5280', '5285', '5288', '5371', '5376', '5458', '5476', '5498', '5507', '5508', '5542', '5561', '5570', '5684', '5692', '5717', '5752', '5779', '5785', '5786', '5810', '5850', '5852', '5895', '5908', '5939', '5941', '5954', '5964', '5973', '5981', '6253', '6406', '6450', '6464', '6535', '6593', '6596', '6637', '6716', '6757', '6866', '6926', '6952', '6954', '6959', '6964', '6976', '7011', '7087', '7135', '718', '7375', '7477', '7521', '7531', '7537', '7543', '7600', '7626', '7628', '7632', '7633', '7640', '7643', '7662', '7771', '7802', '7804', '7907', '8029', '8044', '8080', '8102', '8106', '8132', '8135', '8138', '8141', '8149', '8151', '8153', '8156', '8158', '8173', '8174', '8186', '8189', '8198', '8212', '8240', '8244', '8252', '8273', '843', '8554', '8592', '8596', '8597', '8599', '8602', '8608', '8624', '8635', '8637', '8662', '8663', '8667', '8671', '8676', '8692', '8699', '8725', '8746', '8747', '8911', '8922', '894', '8982', '9007', '9015', '9051', '9055', '9060', '9076', '9135', '9143', '9171', '9184', '9194', '924', '925', '9308', '9315', '9406', '9415', '9493', '9498', '9515', '9532', '9592', '9600', '9603', '9628', '9657', '9697', '9704', '9712', '9729', '9748', '9749', '9751', '9763', '9767', '9768', '9770', '9777', '9784', '9795', '9816', '9821', '9836', '9847', '9848', '9849', '9864', '9897', '991', '9957'],
    'veg-oil': ['10015', '10175', '10300', '10330', '10758', '10903', '1110', '11437', '11671', '11778', '11840', '11886', '11932', '11953', '1211', '12224', '12372', '12425', '12585', '12746', '12786', '13080', '13738', '14509', '14833', '14840', '14892', '15341', '15500', '15751', '15853', '15871', '15875', '15890', '15906', '15923', '15975', '16080', '16193', '16362', '16752', '17477', '17759', '18150', '1848', '18565', '18599', '18700', '18744', '18781', '19088', '19367', '19534', '19551', '19867', '200', '20232', '20335', '20389', '20461', '20911', '21152', '21497', '235', '2420', '2456', '274', '2975', '313', '320', '332', '3499', '3540', '3950', '4035', '4138', '4199', '4305', '4436', '4650', '4681', '5142', '5152', '5169', '5218', '5285', '5292', '5315', '5391', '5506', '5635', '5655', '5761', '5791', '6', '6058', '6062', '6342', '6344', '6354', '6412', '6421', '6535', '6585', '69', '6954', '6993', '6994', '7037', '7515', '7625', '8003', '8086', '8156', '8159', '833', '8596', '885', '9193', '944', '9550', '9756', '9933', '9982'],
    'wheat': ['10120', '10172', '10175', '10268', '10305', '10406', '10444', '10494', '10519', '10569', '1057', '1067', '1069', '10830', '10882', '1094', '10956', '11012', '11040', '11085', '11091', '11119', '11208', '11224', '11316', '11343', '11380', '11392', '11399', '11415', '11427', '11445', '11608', '11610', '11645', '11655', '11671', '11694', '11815', '11834', '1185', '11885', '11886', '12002', '12009', '12192', '12254', '12261', '12269', '12325', '12372', '12420', '12428', '12458', '12500', '12543', '1257', '12762', '12802', '12887', '1295', '12971', '13094', '13097', '13179', '13272', '13313', '13729', '13744', '1377', '1385', '13852', '13856', '1388', '1396', '1405', '14126', '14491', '14509', '14718', '14721', '14722', '14841', '15043', '15097', '15132', '15271', '15273', '15341', '15388', '15472', '15500', '15567', '15572', '15582', '15618', '15676', '15686', '1570', '15728', '15836', '15845', '15853', '15860', '15863', '15864', '15866', '15868', '15869', '15870', '15874', '15875', '15890', '15893', '15895', '1590', '15906', '15911', '15916', '15917', '15927', '15952', '16071', '16144', '16271', '1640', '16525', '1674', '17503', '17521', '17662', '17722', '17755', '1777', '1792', '17955', '17962', '180', '18275', '18297', '18345', '18899', '18908', '19', '1901', '1907', '19169', '19213', '19275', '19322', '19549', '19668', '1970', '19821', '2003', '2008', '20231', '20239', '20258', '2044', '20511', '20550', '20611', '20645', '20738', '2074', '2095', '21091', '21243', '2169', '2172', '2191', '2217', '2223', '235', '2367', '2382', '241', '2425', '2508', '2524', '253', '2535', '254', '2741', '2747', '2857', '2864', '3132', '3138', '3272', '3282', '3299', '3314', '3323', '3330', '3334', '3335', '3358', '3401', '3429', '3445', '3737', '3897', '3904', '3913', '395', '4057', '4077', '4203', '425', '4278', '4296', '4356', '4382', '4473', '4478', '4490', '4549', '4599', '4671', '4696', '5', '501', '5056', '5383', '5467', '5531', '5606', '5610', '5636', '5639', '5648', '6', '6006', '6079', '6081', '6114', '6127', '6153', '6209', '6221', '6239', '6259', '6296', '6326', '6339', '6720', '6744', '6751', '6882', '6898', '6903', '6909', '694', '6946', '6960', '7062', '7205', '7215', '7326', '742', '7471', '7545', '7934', '7957', '8003', '8179', '8273', '8413', '8535', '856', '8604', '874', '8759', '8993', '9021', '9095', '97', '9773', '9782', '9793', '9865'],
    'wpi': ['12429', '13232', '16118', '16164', '16195', '16270', '16746', '16772', '16854', '18366', '18509', '19082', '1965', '4634', '4641', '4660', '4743', '4744', '4787', '4944', '5037', '5061', '5768', '5769', '5839', '6371', '6975', '7585', '8694'],
    'yen': ['10364', '10679', '10681', '10684', '10689', '10696', '10718', '10762', '10766', '10769', '10770', '10804', '11203', '11254', '11764', '11772', '12145', '12470', '13544', '14767', '14913', '15400', '15432', '15454', '15455', '15483', '15503', '15549', '18363', '18370', '19061', '1926', '20862', '21542', '21573', '2178', '2190', '2286', '2354', '3419', '3421', '3532', '4633', '4675', '4680', '4703', '4709', '5204', '5206', '5271', '6338', '6357', '872', '9149', '9213', '9222', '9698', '9701', '9946'],
    'zinc': ['10485', '1077', '11190', '12484', '13877', '14572', '14882', '16194', '17480', '17486', '17783', '17805', '18337', '18943', '18945', '19431', '19477', '19692', '19764', '2186', '3104', '3163', '3183', '3207', '3454', '3862', '5153', '6451', '6995', '7775', '7842', '8308', '8854', '8877'],
}

In [33]:
#Función para calcular precisión y recall utilizando el diccionario de documentos relevantes
def calculate_precision_recall_using_dict(num_relevant_documents, documentos_relevantes):
    precision_values = []
    recall_values = []

    for term in documentos_relevantes:
        relevant_docs = set(documentos_relevantes[term])
        relevant_count = sum(num_relevant_documents[int(doc_id)] >= threshold for doc_id in relevant_docs if int(doc_id) < len(num_relevant_documents))
        retrieved_count = len(relevant_docs)

        # Calcular precisión
        if retrieved_count > 0:
            precision = relevant_count / retrieved_count
        else:
            precision = 0.0
        
        # Calcular recall
        total_relevant = len(relevant_docs)
        if total_relevant > 0:
            recall = relevant_count / total_relevant
        else:
            recall = 0.0
        
        precision_values.append(precision)
        recall_values.append(recall)
    
    return precision_values, recall_values

# Suponiendo que tienes num_relevant_documents calculado previamente
# Aquí deberías reemplazar threshold con tu umbral específico de similitud de Jaccard
threshold = 0.03
precision_values, recall_values = calculate_precision_recall_using_dict(num_relevant_documents, documentos_relevantes)

# Imprimir resultados
for i, term in enumerate(documentos_relevantes):
    print(f"Término '{term}': Precisión = {precision_values[i]:.10f}, Recall = {recall_values[i]:.2f}")

Término 'acq': Precisión = 0.0016884762, Recall = 0.00
Término 'alum': Precisión = 0.0000000000, Recall = 0.00
Término 'barley': Precisión = 0.0000000000, Recall = 0.00
Término 'bop': Precisión = 0.0000000000, Recall = 0.00
Término 'carcass': Precisión = 0.0000000000, Recall = 0.00
Término 'castor-oil': Precisión = 0.0000000000, Recall = 0.00
Término 'cocoa': Precisión = 0.0000000000, Recall = 0.00
Término 'coconut': Precisión = 0.0000000000, Recall = 0.00
Término 'coconut-oil': Precisión = 0.0000000000, Recall = 0.00
Término 'coffee': Precisión = 0.0143884892, Recall = 0.01
Término 'copper': Precisión = 0.0153846154, Recall = 0.02
Término 'copra-cake': Precisión = 0.0000000000, Recall = 0.00
Término 'corn': Precisión = 0.0042194093, Recall = 0.00
Término 'cotton': Precisión = 0.0000000000, Recall = 0.00
Término 'cotton-oil': Precisión = 0.0000000000, Recall = 0.00
Término 'cpi': Precisión = 0.0000000000, Recall = 0.00
Término 'cpu': Precisión = 0.0000000000, Recall = 0.00
Término 'cru

In [34]:
def calcular_promedios(precision_recall):
    precision_promedio = np.mean([prec for prec, _ in precision_recall])
    recall_promedio = np.mean([rec for _, rec in precision_recall])
    return precision_promedio, recall_promedio

In [35]:
# Suponiendo que ya tienes precision_values y recall_values calculados
precision_recall = list(zip(precision_values, recall_values))

precision_promedio, recall_promedio = calcular_promedios(precision_recall)

print(f"Precisión promedio: {precision_promedio:.20f}")
print(f"Recall promedio: {recall_promedio:.23f}")

Precisión promedio: 0.01045357585812089520
Recall promedio: 0.01045357585812089519506


### TF IDF

#### Coseno

In [21]:
# Inicializar el vectorizador TF-IDF
vectorizer_tfIdf = TfidfVectorizer()

# Vectorizar los contenidos preprocesados
X_tfIdf = vectorizer_tfIdf.fit_transform(preprocessed_contents).toarray()

In [22]:
# Vectorizar consultas
querys_of_path_dict = {query: 1 for query in querys_}  # Asumiendo que cada consulta tiene un valor de 1
X_querys_tfIdf = vectorizer_tfIdf.transform(list(querys_of_path_dict.keys())).toarray()

In [23]:
def calculate_cosine_similarities(query_vector, document_vectors):
    similarities = cosine_similarity(query_vector, document_vectors)
    return similarities.flatten()

In [24]:
# Calcular similitudes de coseno para cada consulta
results_tfIdf = []
for i, query_vector in enumerate(X_querys_tfIdf):
    query_vector_2d = query_vector.reshape(1, -1)  # Convertir a 2D
    similarities_tfIdF = calculate_cosine_similarities(query_vector_2d, X_tfIdf)
    results_tfIdf.append(similarities_tfIdF)

# Crear DataFrame con los resultados y nombres de archivo
df_tfIdf = pd.DataFrame(results_tfIdf, columns=file_names)
df_tfIdf.index = querys_

KeyboardInterrupt: 

In [ ]:
print(df_tfIdf)

# Metricas

#### Documentos recuperados

##### BoW - Jaccard

In [ ]:
# Obtener índices de documentos relevantes para cada consulta
documents_recuperate_BoW  = []
umbral = 0.5  # Umbral ajustable según necesidad
cant_doc_rec_BoW = []

In [ ]:
for similarities in results_BoW:
    similarities_array = np.array(similarities)  # Convertir a array de NumPy
    relevant_indices = np.where(similarities_array > umbral)[0]
    cant_doc_rec_BoW.append(len(relevant_indices))

In [ ]:
print(cant_doc_rec_BoW)

In [ ]:
documents_relevantes_por_consulta = []

for similarities in results_BoW:
    similarities_array = np.array(similarities)  # Convertir a array de NumPy
    relevant_indices = np.where(similarities_array > umbral)[0]
    documents_relevantes_por_consulta.append(list(relevant_indices))  # Guardar los índices como lista

print(documents_relevantes_por_consulta)

##### TF IDF - Coseno

In [ ]:
# Obtener índices de documentos relevantes para cada consulta
documents_recuperate_TfIdf  = []
threshold = 0.5  # Umbral ajustable según necesidad

for similarities in results_tfIdf:
    relevant_indices = np.where(similarities > threshold)[0]
    documents_recuperate_TfIdf.append(relevant_indices)

In [ ]:
# Mostrar los resultados
for i, indices in enumerate(documents_recuperate_TfIdf):
    query = querys_[i]
    recuperate_documents_TfIdf = [file_names[idx] for idx in indices]
    print(f"Documentos recuperados para la consulta '{query}':")
    print(recuperate_documents_TfIdf)

In [ ]:
# Convertir resultados a nombres de documentos recuperados
def convertir_a_documentos_recuperados(results, umbral):
    resultados_recuperados = []
    for similitudes in results:
        documentos_recuperados = [idx for idx, sim in enumerate(similitudes) if sim > umbral]
        resultados_recuperados.append(documentos_recuperados)
    return resultados_recuperados

# Definir umbral
umbralT = 0.5

# Convertir resultados a nombres de documentos recuperados
resultados_recuperados_BoW = convertir_a_documentos_recuperados(results_BoW, umbralT)
resultados_recuperados_tfIdf = convertir_a_documentos_recuperados(results_tfIdf, umbralT)

In [ ]:
def calcular_precision_recall(documentos_relevantes, resultados_recuperados):
    precision_recall = {}

    for consulta, relevantes in documentos_relevantes.items():
        consulta_numerica = int(consulta.split('consulta')[-1])  # Extraer el número de la consulta
        recuperados = resultados_recuperados[consulta_numerica - 1]

        # Calcular Precision
        if len(recuperados) > 0:
            precision = len(set(relevantes).intersection(set(recuperados))) / len(recuperados)
        else:
            precision = 0.0

        # Calcular Recall
        if len(relevantes) > 0:
            recall = len(set(relevantes).intersection(set(recuperados))) / len(relevantes)
        else:
            recall = 0.0

        precision_recall[consulta] = {
            'Precision': precision,
            'Recall': recall
        }

    return precision_recall


In [ ]:
# Calcular precisión y recall para BoW y tfIdf
precision_recall_BoW = calcular_precision_recall(documentos_relevantes, resultados_recuperados_BoW)
precision_recall_tfIdf = calcular_precision_recall(documentos_relevantes, resultados_recuperados_tfIdf)

# Calcular promedios
def calcular_promedios(precision_recall):
    precision_promedio = np.mean([precision_recall[consulta]['Precision'] for consulta in precision_recall.keys()])
    recall_promedio = np.mean([precision_recall[consulta]['Recall'] for consulta in precision_recall.keys()])
    return precision_promedio, recall_promedio

precision_promedio_BoW, recall_promedio_BoW = calcular_promedios(precision_recall_BoW)
precision_promedio_tfIdf, recall_promedio_tfIdf = calcular_promedios(precision_recall_tfIdf)

print(f"Precisión promedio BoW: {precision_promedio_BoW}")
print(f"Recall promedio BoW: {recall_promedio_BoW}")
print(f"Precisión promedio tfIdf: {precision_promedio_tfIdf}")
print(f"Recall promedio tfIdf: {recall_promedio_tfIdf}")